In [ ]:
# ---- Agent tools: dummy create + update-by-id via activities/run ----
import os
import json
from typing import Any, Dict
import httpx

try:
    from agents import function_tool
except Exception:
    # Fallback no-op decorator if not available at import-time
    def function_tool(func=None, **_kwargs):
        return func if func else (lambda f: f)

ACTIVITY_URL_APP = "https://devdailyservice.kahua.com/v2/domains/AWrightCo/projects/{id}/apps/kahua_AEC_RFI/activities/run"


def _auth_header_env() -> str:
    token = os.getenv("KAHUA_BASIC_AUTH")
    if not token:
        raise RuntimeError("KAHUA_BASIC_AUTH not set")
    return token if token.strip().lower().startswith("basic ") else f"Basic {token}"


def _activity_url_by_app(app_key: str, projectid: int | None) -> str:
    pid = projectid if projectid is not None else 0
    return ACTIVITY_URL_APP.format(id=pid, app=app_key)


def _extract_first_entity(data: Dict[str, Any], set_name: str) -> Dict[str, Any]:
    sets = data.get("sets") or []
    for s in sets:
        if s.get("name") == set_name:
            ents = s.get("entities") or []
            if ents:
                return ents[0]
    for key in ("entities", "items", "results"):
        if isinstance(data.get(key), list) and data[key]:
            return data[key][0]
    return {}


@function_tool(strict_mode=False)
async def create_dummy_entity(
    entity_def: str,
    app_key: str = "kahua_AEC_RFI",
    projectid: int | None = None,
    set_name: str = "Activity",
) -> dict:
    """
    Create a dummy entity (id=0) via activities/run to retrieve the full schema in the response.
    Returns: { status, upstream_status, kahua_response, created_entity }
    """
    headers = {"Content-Type": "application/json", "Authorization": _auth_header_env()}
    payload = {
        "activity": {
            "PropertyName": set_name,
            "Name": set_name,
            "Flow": [
                {"PropertyName": "Iterate", "Set": set_name, "New": {}, "Existing": {}}
            ],
        },
        "listIdentifier": 0,
        "listId": 0,
        "sets": [
            {
                "name": set_name,
                "entities": [
                    {
                        "id": 0,
                        "entityDef": entity_def,
                    }
                ],
            }
        ],
    }

    url = _activity_url_by_app(app_key, projectid)
    async with httpx.AsyncClient(timeout=20.0) as client:
        resp = await client.post(url, headers=headers, json=payload)
        ctype = resp.headers.get("content-type", "")
        body = resp.json() if "application/json" in (ctype or "") else {"text": resp.text}

    if resp.status_code >= 400:
        return {"status": "error", "upstream_status": resp.status_code, "upstream_body": body}

    created = _extract_first_entity(body, set_name)
    return {
        "status": "ok",
        "upstream_status": resp.status_code,
        "kahua_response": body,
        "created_entity": created,
    }


@function_tool(strict_mode=False)
async def update_entity_by_id(
    entity_def: str,
    entity_id: int,
    fields: Dict[str, Any] | None = None,
    app_key: str = "kahua_AEC_RFI",
    projectid: int | None = None,
    set_name: str = "Activity",
) -> dict:
    """
    Update an existing entity by re-posting with the same id. `fields` should contain
    field-name to value mappings according to the schema returned from create.
    Returns: { status, upstream_status, kahua_response }
    """
    headers = {"Content-Type": "application/json", "Authorization": _auth_header_env()}
    entity: Dict[str, Any] = {"id": int(entity_id), "entityDef": entity_def}
    if fields:
        entity.update(dict(fields))

    payload = {
        "activity": {
            "PropertyName": set_name,
            "Name": set_name,
            "Flow": [
                {"PropertyName": "Iterate", "Set": set_name, "New": {}, "Existing": {}}
            ],
        },
        "listIdentifier": 0,
        "listId": 0,
        "sets": [
            {
                "name": set_name,
                "entities": [entity],
            }
        ],
    }

    url = _activity_url_by_app(app_key, projectid)
    async with httpx.AsyncClient(timeout=20.0) as client:
        resp = await client.post(url, headers=headers, json=payload)
        ctype = resp.headers.get("content-type", "")
        body = resp.json() if "application/json" in (ctype or "") else {"text": resp.text}

    if resp.status_code >= 400:
        return {"status": "error", "upstream_status": resp.status_code, "upstream_body": body}

    return {"status": "ok", "upstream_status": resp.status_code, "kahua_response": body}

# Wire these tools into the Custom Item agent so the router can use them
try:
    custom_item_agent.tools = [create_punch_list_item, send_custom_item, create_dummy_entity, update_entity_by_id]
except Exception:
    pass

In [ ]:
# ---- Agent behavior tweaks: prefer tool calls over asking user for schema ----
try:
    # Expand aliases (singular/plural/short) for better intent matching
    ENTITY_ALIASES.update({
        "field observation": "kahua_AEC_FieldObservation.FieldObservationItem",
        "fo": "kahua_AEC_FieldObservation.FieldObservationItem",
        "rfi": "kahua_AEC_RFI.RFI",
        "submittal": "kahua_AEC_Submittal.Submittal",
    })
except Exception:
    pass

# Helper: derive app_key from entity_def prefix (e.g., "kahua_AEC_FieldObservation.FieldObservationItem" -> "kahua_AEC_FieldObservation")
def _app_key_from_entity_def(entity_def: str) -> str:
    return entity_def.split(".")[0] if isinstance(entity_def, str) and "." in entity_def else "kahua_AEC_RFI"

try:
    # Instruct Custom Item agent to auto-fetch schema via dummy create, then update
    custom_item_agent.instructions = (
        (custom_item_agent.instructions or "")
        + "\n\nStrict policy for Kahua entity creation (Field Observation, RFI, Submittal, etc.):\n"
        + "1) Do NOT ask the user for a schema.\n"
        + "2) Map alias → entityDef using ENTITY_ALIASES. Set app_key to the prefix of entityDef before the dot.\n"
        + "3) Call create_dummy_entity(entity_def, app_key, projectid if known) to obtain id + schema.\n"
        + "4) Choose fields from the user's intent (at minimum set 'Subject' if appropriate).\n"
        + "5) Call update_entity_by_id(entity_def, entity_id, fields, app_key, projectid) to persist edits on the created item.\n"
        + "6) Return a concise JSON result with id, fields set, and any relevant links.\n"
    )
    # Ensure tools are present
    custom_item_agent.tools = list({t for t in (custom_item_agent.tools or [])} | {create_punch_list_item, send_custom_item, create_dummy_entity, update_entity_by_id})
except Exception:
    pass

try:
    # Nudge triage to hand off creation requests to Custom Item agent
    triage_agent.instructions = (
        (triage_agent.instructions or "")
        + "\n- If the user asks to create a Kahua entity (e.g., Field Observation, RFI, Submittal), hand off to Custom Item.\n"
        + "  Custom Item must auto-call create_dummy_entity then update_entity_by_id — do not ask the user for schema.\n"
    )
except Exception:
    pass

In [507]:
import os, json
import httpx
from typing import Optional, List, Any, Dict, Type
from pydantic import BaseModel, Field, create_model, ValidationError
from openai import AsyncAzureOpenAI
from agents import Agent, Runner, OpenAIChatCompletionsModel, function_tool
import logging
# -------- Tool: post the punch item to your route --------
# Send extracted punch list to Kahua

# Global logging configuration
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(name)s: %(message)s")
logger = logging.getLogger("kahua_tool")
logger.setLevel(logging.INFO)
# Dedicated logger for agent handoffs
handoff_log = logging.getLogger("agents.handoff")
handoff_log.setLevel(logging.INFO)

In [508]:
# -------- Azure OpenAI client --------
azure_client = AsyncAzureOpenAI(
    api_key=os.environ["AZURE_KEY"],
    azure_endpoint=os.environ["AZURE_ENDPOINT"],
    api_version=os.environ["API_VERSION"],
)

In [ ]:
# -------- Schema the agent will produce --------
class PunchItem(BaseModel):
    subject: Optional[str] = Field(None, description="Short subject")
    description: Optional[str] = Field(None, description="Detailed description")
    projectid: Optional[int] = Field(0, description="Project id")
    # Optional flag so the agent can decide to actually send
    auto_send: bool = Field(False, description="If true, call the Kahua tool")

class ProjectItem(BaseModel):
    id: Optional[str] = Field(0, description="Project id")
    name: str = Field(..., description="Project name (Kahua: Name)")
    description: Optional[str] = Field(None, description="Project description (Kahua: Description)")
    auto_send: bool = Field(True, description="If true, tool can be called immediately by the agent")

In [ ]:
def _auth_header() -> str:
    if not KAHUA_BASIC_AUTH:
        raise RuntimeError("KAHUA_BASIC_AUTH is not set in environment.")
    return KAHUA_BASIC_AUTH if KAHUA_BASIC_AUTH.strip().lower().startswith("basic ") else f"Basic {KAHUA_BASIC_AUTH}"

def _redact(auth: str) -> str:
    try:
        return (auth[:10] + "...redacted") if isinstance(auth, str) else "<none>"
    except Exception:
        return "<redacted>"

In [511]:
KAHUA_BASIC_AUTH = os.getenv("KAHUA_BASIC_AUTH")
ACTIVITY_URL = "https://devdailyservice.kahua.com/v2/domains/AWrightCo/projects/{id}/apps/kahua_AEC_RFI/activities/run"
QUERY_URL = "https://devdailyservice.kahua.com/v2/domains/AWrightCo/projects/0/query?returnDefaultAttributes=true"

In [512]:
@function_tool
async def create_project_in_kahua(project: ProjectItem) -> dict:
    """
    Create a Project in Kahua (entityDef: kahua_Project.Project).
    Mirrors your requests-based example, but as an async Agents tool.
    """
    # Map ProjectItem -> Kahua fields
    entity = {
        "id": 0,
        "hubPath": "kahua_Project.NoWorkflow\\Start",
        "entityDef": "kahua_Project.Project",
        "Name": project.name,
    }
    if project.description:
        entity["Description"] = project.description
    if project.id:
        entity["Id"] = project.id

    payload = {
        "activity": {
            "PropertyName": "Activity",
            "Name": "ContractItem",
            "Flow": [
                {"PropertyName": "Iterate", "Set": "ContractItem", "New": {}, "Existing": {}}
            ],
        },
        "sets": [{"name": "ContractItem", "entities": [entity]}],
    }

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Basic {KAHUA_BASIC_AUTH}",
    }
    # Redacted log
    try:
        redacted = (KAHUA_BASIC_AUTH or "")[:10] + "...redacted" if isinstance(KAHUA_BASIC_AUTH, str) else "<none>"
        logger.info(f"POST {ACTIVITY_URL.format(id=0)} | Auth={redacted}")
        logger.info(f"Payload preview: {json.dumps({'Name': entity.get('Name'), 'Description': (entity.get('Description') or '')[:120]}, ensure_ascii=False)}")
    except Exception:
        pass

    async with httpx.AsyncClient(timeout=20.0) as client:
        resp = await client.post(ACTIVITY_URL.format(id=0), headers=headers, json=payload)
        ctype = resp.headers.get("content-type", "")
        body = resp.json() if "application/json" in (ctype or "") else {"text": resp.text}

        if resp.status_code >= 400:
            logger.error(f"Kahua project create failed {resp.status_code}: {resp.text}")
            return {
                "status": "error",
                "upstream_status": resp.status_code,
                "upstream_content_type": ctype,
                "upstream_body": body,
                "payload_preview": {"Name": entity.get("Name"), "Description": (entity.get("Description") or "")[:200]},
            }

        return {
            "status": "ok",
            "upstream_status": resp.status_code,
            "kahua_response": body,
            "payload_preview": {"Name": entity.get("Name"), "Description": (entity.get("Description") or "")[:200]},
        }

In [513]:
@function_tool
async def find_project_id_by_name(name: str) -> dict:
    """
    Look up a project's Id by exact or fuzzy Name match using the query endpoint.
    Returns: { status, matches: [ {Id, Name, ...} ], best_match: {Id, Name} | None }
    """
    if not name or not isinstance(name, str):
        return {"status": "error", "message": "name must be a non-empty string"}

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Basic {KAHUA_BASIC_AUTH}",
    }

    # Base query for Projects
    base_payload = {
        "PropertyName": "Query",
        "EntityDef": "kahua_Project.Project",
    }

    async with httpx.AsyncClient(timeout=20.0) as client:
        # 1) Try exact match on Name
        exact_payload = dict(base_payload)
        exact_payload["Filter"] = {
            "PropertyName": "Filter",
            "And": [
                {"PropertyName": "Equals", "Field": "Name", "Value": name}
            ]
        }
        resp = await client.post(QUERY_URL, headers=headers, json=exact_payload)
        ctype = resp.headers.get("content-type", "")
        body = resp.json() if "application/json" in (ctype or "") else {"text": resp.text}
        if resp.status_code == 200 and isinstance(body, dict) and body.get("sets"):
            rows = (body.get("sets", [{}])[0].get("entities") or [])
            matches = [ {"Id": r.get("Id"), "Name": r.get("Name")} for r in rows ]
            if matches:
                return {"status": "ok", "matches": matches, "best_match": matches[0]}

        # 2) Fallback: contains/like on Name
        like_payload = dict(base_payload)
        like_payload["Filter"] = {
            "PropertyName": "Filter",
            "And": [
                {"PropertyName": "Contains", "Field": "Name", "Value": name}
            ]
        }
        resp2 = await client.post(QUERY_URL, headers=headers, json=like_payload)
        ctype2 = resp2.headers.get("content-type", "")
        body2 = resp2.json() if "application/json" in (ctype2 or "") else {"text": resp2.text}
        if resp2.status_code == 200 and isinstance(body2, dict) and body2.get("sets"):
            rows = (body2.get("sets", [{}])[0].get("entities") or [])
            matches = [ {"Id": r.get("Id"), "Name": r.get("Name")} for r in rows ]
            best = matches[0] if matches else None
            return {"status": "ok", "matches": matches, "best_match": best}

        return {"status": "error", "message": "No matches found", "artifacts": {"exact": body, "like": body2 if 'body2' in locals() else None}}


In [514]:
@function_tool
async def query_entity_def(entity_def: str) -> dict:
    url = QUERY_URL
    payload = {
        "PropertyName": "Query",
        "EntityDef": entity_def,
    }
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Basic {KAHUA_BASIC_AUTH}",
    }

    async with httpx.AsyncClient(timeout=20.0) as client:
        resp = await client.post(url, headers=headers, json=payload)
        ctype = resp.headers.get("content-type", "")
        body = resp.json() if "application/json" in (ctype or "") else {"text": resp.text}

        if resp.status_code >= 400:
            return {
                "status": "error",
                "upstream_status": resp.status_code,
                "upstream_content_type": ctype,
                "upstream_body": body,
            }

        return {
            "status": "ok",
            "upstream_status": resp.status_code,
            "body": body,
        }

In [515]:
@function_tool(strict_mode=False)
async def send_custom_item(entity: Dict[str, Any], projectid: int | None = None) -> dict:
    """
    Send an arbitrary entity payload you've already shaped.
    Requirements:
      - entityDef (e.g., "kahua_AEC_Submittal.Submittal")
    """
    if not isinstance(entity, dict):
        return {"status": "error", "message": "entity must be a dict"}
    if not entity.get("entityDef"):
        return {"status": "error", "message": "entityDef is required"}

    entity = dict(entity)
    entity["id"] = 0

    payload = {
        "activity": {
            "PropertyName": "Activity",
            "Name": "ContractItem",
            "Flow": [{"PropertyName": "Iterate", "Set": "ContractItem", "New": {}, "Existing": {}}]
        },
        "sets": [{"name": "ContractItem", "entities": [entity]}],
    }

    pid = projectid if projectid is not None else entity.get("ProjectId") or 0
    url = ACTIVITY_URL.format(id=pid)
    auth = _auth_header()
    headers = {"Content-Type": "application/json", "Authorization": auth}

    async with httpx.AsyncClient(timeout=20.0) as client:
        resp = await client.post(url, headers=headers, json=payload)
        ctype = resp.headers.get("content-type", "")
        body = resp.json() if "application/json" in (ctype or "") else {"text": resp.text}

        if resp.status_code >= 400:
            return {"status": "error", "upstream_status": resp.status_code, "upstream_body": body}

        return {"status": "ok", "upstream_status": resp.status_code, "kahua_response": body}


In [ ]:
def _pick_representative(lst: list) -> Any:
    for x in lst:
        if x is not None:
            return x
    return lst[0] if lst else None

def _infer_model_from_example(data: Any, name_hint: str = "Model") -> Type[BaseModel] | Any:
    if isinstance(data, bool):   return Optional[bool]
    if isinstance(data, int):    return Optional[int]
    if isinstance(data, float):  return Optional[float]
    if isinstance(data, str):    return Optional[str]
    if data is None:             return Optional[Any]

    if isinstance(data, list):
        rep = _pick_representative(data)
        if isinstance(rep, dict):
            sub = _infer_model_from_example(rep, f"{name_hint}Item")
            return Optional[List[sub]]  # type: ignore[name-defined]
        elif rep is None:
            return Optional[List[Any]]
        else:
            prim = _infer_model_from_example(rep, f"{name_hint}Item")
            return Optional[List[prim]]  # type: ignore[valid-type]

    if isinstance(data, dict):
        fields: Dict[str, tuple] = {}
        for k, v in data.items():
            sub_type = _infer_model_from_example(v, f"{name_hint}_{k.capitalize()}")
            fields[k] = (sub_type, None)
        return create_model(name_hint, **fields)  # type: ignore[arg-type]

    return Optional[Any]

def _sanitize_json_schema(obj: Any) -> Any:
    if isinstance(obj, dict):
        obj.pop("additionalProperties", None)
        for k, v in list(obj.items()):
            obj[k] = _sanitize_json_schema(v)
    elif isinstance(obj, list):
        return [_sanitize_json_schema(x) for x in obj]
    return obj

@function_tool(strict_mode=False)
async def create_pydantic_model_from_schema(
    schema_like: Dict[str, Any],
    model_name: str = "DynamicModel",
    sample: Optional[Dict[str, Any]] = None
) -> dict:
    try:
        ModelOrType = _infer_model_from_example(schema_like, model_name)
        if not isinstance(ModelOrType, type) or not issubclass(ModelOrType, BaseModel):
            Model = create_model(model_name, value=(ModelOrType, None))  # type: ignore[arg-type]
        else:
            Model = ModelOrType

        json_schema = _sanitize_json_schema(Model.model_json_schema())

        validated_sample = None
        errors: List[str] = []
        status = "ok"

        if sample is not None:
            try:
                inst = Model(**sample)
                validated_sample = inst.model_dump(by_alias=False, exclude_none=True)
            except ValidationError as ve:
                status = "validation_error"
                errors = [str(e) for e in ve.errors()]

        try:
            python_model_str = f"class {Model.__name__}(BaseModel): " + ", ".join(Model.__fields__.keys())  # type: ignore[attr-defined]
        except Exception:
            python_model_str = f"{Model.__name__}(BaseModel)"

        return {
            "status": status,
            "model_name": Model.__name__,
            "json_schema": json_schema,
            "python_model_str": python_model_str,
            "validated_sample": validated_sample,
            "errors": errors,
        }
    except Exception as e:
        return {
            "status": "error",
            "model_name": model_name,
            "json_schema": None,
            "python_model_str": None,
            "validated_sample": None,
            "errors": [repr(e)],
        }

In [ ]:
# ---- Tool: post directly to Kahua (no FastAPI bridge) ----
@function_tool
async def create_punch_list_item(item: PunchItem) -> dict:
    """
    Create a PunchListItem in Kahua using the same logic as your FastAPI route.
    Expects env:
      - KAHUA_URL (optional; default points to devdailyservice)
      - KAHUA_BASIC_AUTH (Base64 Basic header, e.g., 'Basic <base64userpass>')
    """
    logger.info("Sending to Kahua (tool)")

    # Convert to plain dict so we can check alternate keys if you add any later
    data = item.model_dump()
    if not data.get("projectid"):
        data["projectid"] = 0

    # --- Subject/Description synthesis (mirrors your server-side logic) ---
    subject = data.get("subject")
    description = data.get("description")

    if not subject or not description:
        location = data.get("location") or data.get("area")
        defect = data.get("defect")
        desc = data.get("description")
        if not subject:
            if location and defect:
                subject = f"{location} – {defect}"
            elif location and desc:
                subject = f"{location} – {desc[:60]}"
            else:
                subject = defect or (desc[:80] if desc else "Punch Item")
        if not description:
            description = desc or defect or subject

    if not subject or not description:
        logger.warning("Kahua export missing subject/description; using synthesized fallbacks")
        subject = subject or "Punch Item"
        description = description or subject

    kahua_auth = f"Basic {KAHUA_BASIC_AUTH}"
    # --- Payload mirrors your route ---
    payload = {
        "activity": {
            "PropertyName": "Activity",
            "Name": "ContractItem",
            "Flow": [
                {
                    "PropertyName": "Iterate",
                    "Set": "ContractItem",
                    "New": {},
                    "Existing": {},
                }
            ],
        },
        "sets": [
            {
                "name": "ContractItem",
                "entities": [
                    {
                        "id": 0,
                        "hubPath": "kahua_AEC_PunchList.NoWorkflow\\Start",
                        "entityDef": "kahua_AEC_PunchList.PunchListItem",
                        "Subject": subject,
                        "Description": description,
                    }
                ],
            }
        ],
    }

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Basic {KAHUA_BASIC_AUTH}",
    }   

    # Sanitized logs
    try:
        redacted = kahua_auth[:10] + "...redacted" if isinstance(kahua_auth, str) else "<none>"
        logger.info(f"Kahua URL: {ACTIVITY_URL.format(id=data.get('projectid', 0))} | Auth: {redacted}")
        logger.info(f"Kahua payload preview: {json.dumps({'Subject': subject, 'Description': (description or '')[:200]}, ensure_ascii=False)}")
    except Exception:
        pass

    # --- Send ---
    async with httpx.AsyncClient(timeout=20.0) as client:
        resp = await client.post(ACTIVITY_URL.format(id=data.get('projectid', 0)), headers=headers, json=payload)
        ctype = resp.headers.get("content-type", "")
        text_body = resp.text
        logger.info(f"Kahua response {resp.status_code} | content-type={ctype}")

        if resp.status_code >= 400:
            logger.error(f"Kahua send failed {resp.status_code}: {text_body}")
            return {
                "status": "error",
                "upstream_status": resp.status_code,
                "upstream_content_type": ctype,
                "upstream_body": text_body,
                "payload_preview": {"Subject": subject, "Description": description[:200] if description else None},
            }

        data = resp.json() if "application/json" in ctype else {"text": text_body}
        return {
            "status": "ok",
            "upstream_status": resp.status_code,
            "kahua_response": data,
            "payload_preview": {"Subject": subject, "Description": description[:200] if description else None},
        }

In [ ]:
# ---- Tool: log agent handoffs ----
@function_tool
async def log_handoff(from_agent: str, to_agent: str, reason: str = "") -> dict:
    handoff_log.info(f"HANDOFF: {from_agent} -> {to_agent} | reason={reason}")
    return {"status": "logged", "from": from_agent, "to": to_agent, "reason": reason}

In [ ]:
# --- entity_aliases.py ---
ENTITY_ALIASES = {
    # Core
    "project": "kahua_Project.Project",
    "punchlist": "kahua_AEC_PunchList.PunchListItem",
    "rfi": "kahua_AEC_RFI.RFI",
    "submittal": "kahua_AEC_Submittal.Submittal",
    "change order": "kahua_AEC_ChangeOrder.ChangeOrder",
    "field observations": "kahua_AEC_FieldObservation.FieldObservationItem",
}

entity_query_agent = Agent(
    name="Entity Query",
    handoff_description="Specialist agent for entity query",
    model=OpenAIChatCompletionsModel(
        model=os.environ["AZURE_DEPLOYMENT"], openai_client=azure_client
    ),
    instructions=(
        "You can either look up an entityDef, OR find a project id by name.\n"
        "- To find a project id by name, call find_project_id_by_name(name).\n"
        "- Otherwise, to query any entity, map alias → entity_def and call query_entity_def(entity_def).\n\n"
        "ALIAS TABLE (authoritative):\n"
        + "\n".join([f"- {k} → {v}" for k,v in ENTITY_ALIASES.items()]) + "\n\n"
    ),
    tools=[query_entity_def, find_project_id_by_name]
)

project_creation_agent = Agent(
    name="Project",
    handoff_description="Specialist agent for project creation",
    model=OpenAIChatCompletionsModel(
        model=os.environ["AZURE_DEPLOYMENT"],  # <-- deployment name in Azure
        openai_client=azure_client
    ),
    instructions="You help users create construction projects.",
    tools=[create_project_in_kahua]
)

punch_list_agent = Agent(
    name="Punch List",
    handoff_description="Specialist agent for punch list creation.",
    model=OpenAIChatCompletionsModel(
        model=os.environ["AZURE_DEPLOYMENT"],  # <-- deployment name in Azure
        openai_client=azure_client
    ),
    instructions=(
        "You help users create construction punch list items."
    ),
    tools=[create_punch_list_item],
)
# -------- Schema Creation specialist --------
schema_creation_agent = Agent(
    name="Schema Creation",
    handoff_description="Builds dynamic Pydantic models from arbitrary schema-like dicts and (optionally) validates a sample.",
    model=OpenAIChatCompletionsModel(
        model=os.environ["AZURE_DEPLOYMENT"],
        openai_client=azure_client
    ),
    instructions=(
        "You are the Schema Creation specialist.\n"
        "Input to you will be a JSON object with keys:\n"
        "  - schema_like: dict (arbitrary nested structure or example instance)\n"
        "  - model_name: optional string for the generated model\n"
        "  - sample: optional dict to validate against the generated model\n\n"
        "Call the tool create_pydantic_model_from_schema with those fields.\n"
        "Return ONLY the tool's result (JSON). Do not add prose.\n"
    ),
    tools=[create_pydantic_model_from_schema],
)
custom_item_agent = Agent(
    name="Custom Item",
    handoff_description="Specialist agent for custom item creation",
    model=OpenAIChatCompletionsModel(
        model=os.environ["AZURE_DEPLOYMENT"],
        openai_client=azure_client
    ),
    instructions=(
        "You help users create custom items in Kahua. "
    ),
    tools=[send_custom_item],
)
# -------- Router / triage --------
triage_agent = Agent(
    name="Agentic Chatbot",
    instructions=(
        "You are a helpful assistant and router.\n"
        "- If the user asks to create a punch list for a named project, FIRST call find_project_id_by_name(name). If found, pass projectid to Punch List and create exactly once.\n"
        "- If they want a project created, hand off to Project.\n"
        "- If they want to query a Kahua entity, hand off to Entity Query.\n"
        "- If they ask to create/generate a schema, build a Pydantic model from a dict, or validate a sample against a generated model, hand off to Schema Creation.\n"
        "- Avoid repeated handoffs; perform at most one lookup and one create.\n"
        "- Otherwise, answer directly.\n\n"
    ),
    model=OpenAIChatCompletionsModel(
        model=os.environ["AZURE_DEPLOYMENT"],
        openai_client=azure_client
    ),
    # tools=[log_handoff],  # optional: uncomment if you wired log_handoff
    handoffs=[punch_list_agent, project_creation_agent, entity_query_agent, schema_creation_agent, custom_item_agent],
)

In [520]:
from agents import Agent, Runner, SQLiteSession

# Create a session instance with a session ID
session = SQLiteSession("fortyfour")

In [521]:
result = Runner.run_sync(
    triage_agent,
    "create a field observation about the dog i found biting a worker. leave all other fields blank/null. no project id. you have the correct entity def",
    session=session,
)

2025-10-20 13:48:22,919 INFO httpx: HTTP Request: POST https://kai0721300033.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-10-20 13:48:23,310 INFO httpx: HTTP Request: POST https://kai0721300033.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-10-20 13:48:24,009 INFO httpx: HTTP Request: POST https://kai0721300033.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


In [ ]:
print(result.final_output)

# Custom FNs

In [ ]:

# ── Dynamic Pydantic model builder (your “schema” tool) ────────────────────────
def _pick_representative(lst: list) -> Any:
    for x in lst:
        if x is not None:
            return x
    return lst[0] if lst else None

def _infer_model_from_example(data: Any, name_hint: str = "Model") -> Type[BaseModel] | Any:
    if isinstance(data, bool):   return Optional[bool]
    if isinstance(data, int):    return Optional[int]
    if isinstance(data, float):  return Optional[float]
    if isinstance(data, str):    return Optional[str]
    if data is None:             return Optional[Any]

    if isinstance(data, list):
        rep = _pick_representative(data)
        if isinstance(rep, dict):
            sub = _infer_model_from_example(rep, f"{name_hint}Item")
            return Optional[List[sub]]  # type: ignore[name-defined]
        elif rep is None:
            return Optional[List[Any]]
        else:
            prim = _infer_model_from_example(rep, f"{name_hint}Item")
            return Optional[List[prim]]  # type: ignore[valid-type]

    if isinstance(data, dict):
        fields: Dict[str, tuple] = {}
        for k, v in data.items():
            sub_type = _infer_model_from_example(v, f"{name_hint}_{k.capitalize()}")
            fields[k] = (sub_type, None)
        return create_model(name_hint, **fields)  # type: ignore[arg-type]

    return Optional[Any]

def _sanitize_json_schema(obj: Any) -> Any:
    if isinstance(obj, dict):
        obj.pop("additionalProperties", None)
        for k, v in list(obj.items()):
            obj[k] = _sanitize_json_schema(v)
    elif isinstance(obj, list):
        return [_sanitize_json_schema(x) for x in obj]
    return obj

@function_tool(strict_mode=False)
async def create_pydantic_model_from_schema(
    schema_like: Dict[str, Any],
    model_name: str = "DynamicModel",
    sample: Optional[Dict[str, Any]] = None
) -> dict:
    try:
        ModelOrType = _infer_model_from_example(schema_like, model_name)
        if not isinstance(ModelOrType, type) or not issubclass(ModelOrType, BaseModel):
            Model = create_model(model_name, value=(ModelOrType, None))  # type: ignore[arg-type]
        else:
            Model = ModelOrType

        json_schema = _sanitize_json_schema(Model.model_json_schema())

        validated_sample = None
        errors: List[str] = []
        status = "ok"

        if sample is not None:
            try:
                inst = Model(**sample)
                validated_sample = inst.model_dump(by_alias=False, exclude_none=True)
            except ValidationError as ve:
                status = "validation_error"
                errors = [str(e) for e in ve.errors()]

        try:
            python_model_str = f"class {Model.__name__}(BaseModel): " + ", ".join(Model.__fields__.keys())  # type: ignore[attr-defined]
        except Exception:
            python_model_str = f"{Model.__name__}(BaseModel)"

        return {
            "status": status,
            "model_name": Model.__name__,
            "json_schema": json_schema,
            "python_model_str": python_model_str,
            "validated_sample": validated_sample,
            "errors": errors,
        }
    except Exception as e:
        return {
            "status": "error",
            "model_name": model_name,
            "json_schema": None,
            "python_model_str": None,
            "validated_sample": None,
            "errors": [repr(e)],
        }

@function_tool(strict_mode=False)
async def send_custom_item(data: BaseModel) -> dict:
    """
    Send a *custom* entity payload you've already shaped as a Pydantic model.
    """
    entity = data.model_dump()
    entity["id"] = 0
    if not entity.get("entityDef"):
        return {"status": "error", "message": "entityDef is required"}
    payload = {
        "activity": {
            "PropertyName": "Activity",
            "Name": "ContractItem",
            "Flow": [{"PropertyName": "Iterate", "Set": "ContractItem", "New": {}, "Existing": {}}]
        },
        "sets": [{"name": "ContractItem", "entities": [entity]}],
    }

    project_id = getattr(data, "projectid", 0) or entity.get("ProjectId") or 0
    url = URL_TPL.format(id=project_id)
    auth = _auth_header()
    headers = {"Content-Type": "application/json", "Authorization": auth}

    async with httpx.AsyncClient(timeout=20.0) as client:
        resp = await client.post(url, headers=headers, json=payload)
        ctype = resp.headers.get("content-type", "")
        body = resp.json() if "application/json" in (ctype or "") else {"text": resp.text}

        if resp.status_code >= 400:
            logger.error(f"Kahua custom item failed {resp.status_code}: {resp.text}")
            return {
                "status": "error",
                "upstream_status": resp.status_code,
                "upstream_content_type": ctype,
                "upstream_body": body,
            }

        return {"status": "ok", "upstream_status": resp.status_code, "kahua_response": body}

In [4]:
import os
from openai import AzureOpenAI

endpoint = "https://kai0721300033.cognitiveservices.azure.com/"
model_name = "gpt-5-mini"
deployment = "gpt-5-mini"

subscription_key = "3LNEukmYzIfFZdnkIFQZHJqx0hJcGzfse418xSeddfIxFNlgot9EJQQJ99AKACHYHv6XJ3w3AAAAACOGti3b"
api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a mean, unhelpful assistant.",
        },
        {
            "role": "user",
            "content": "What model are you",
        }
    ],
    max_completion_tokens=16384,
    model=deployment
)

print(response.choices[0].message.content)

You're talking to an OpenAI large language model from the GPT‑4 family. I have a knowledge cutoff of June 2024 and the current date is 2025‑10‑27. I don't have extra internal diagnostic info about the exact engine name your app used; if you need the precise model identifier (for example gpt-4o, gpt-4o-mini, etc.), check the app or API call that created this session.


In [44]:
import json, re
from urllib.parse import urlparse

# === CONFIG ===
HAR_FILE = "kahua2.har"   # path to your exported HAR
HOST_FILTER = "devdailyservice.kahua.com"  # match devdailyservice.kahua.com, etc.

# === LOAD & EXTRACT ===
with open(HAR_FILE, "r", encoding="utf-8") as f:
    har = json.load(f)

urls = set()
for e in har["log"]["entries"]:
    url = e["request"]["url"]
    if HOST_FILTER in url:
        # remove query params and normalize
        path = urlparse(url).path
        path = re.sub(r"/\d+(?=/|$)", "/{id}", path)  # replace numeric IDs
        path = re.sub(r"[0-9a-f]{8}-[0-9a-f]{4}-[0-9a-f]{4}-[0-9a-f]{4}-[0-9a-f]{12}", "{guid}", path, flags=re.I)
        urls.add(f"{urlparse(url).netloc}{path}")

# === DISPLAY ===
for u in sorted(urls):
    print(u)

In [45]:
for ix, entry in enumerate(har['log']['entries']):
    if 'contract' in entry['request']['url'].lower():
        print(ix, entry)
        print("-----------------------")

8 {'startedDateTime': '2025-10-30T15:16:35.623-04:00', 'request': {'bodySize': 0, 'method': 'GET', 'url': 'https://devweeklyservice.kahua.com/host/domains/AWrightCo/projects/0/start/app?appGroupName=kahua_Contract&appName=kahua_Contract', 'httpVersion': 'HTTP/1.1', 'headers': [{'name': 'Host', 'value': 'devweeklyservice.kahua.com'}, {'name': 'User-Agent', 'value': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:144.0) Gecko/20100101 Firefox/144.0'}, {'name': 'Accept', 'value': 'application/json'}, {'name': 'Accept-Language', 'value': 'en-US,en;q=0.5'}, {'name': 'Accept-Encoding', 'value': 'gzip, deflate, br, zstd'}, {'name': 'app-name', 'value': 'kahua_Contract'}, {'name': 'authorization', 'value': 'Session 8nLMg0Y-tk6ud4r-4QXGbg'}, {'name': 'domainpartitionid', 'value': '0'}, {'name': 'domainprofilekey', 'value': 'AWrightCo|49861465|1974192'}, {'name': 'requestid', 'value': '6f81e852-5147-4490-bb5c-0a0f7697347b'}, {'name': 'Origin', 'value': 'https://devweeklylaunch.kahua.com'}, {'name'

In [46]:
har['log']['entries'][17]

{'startedDateTime': '2025-10-30T15:16:42.954-04:00',
 'request': {'bodySize': 0,
  'method': 'GET',
  'url': 'https://devweeklylaunch.kahua.com/package_fbd1f792339bff1a48f13aeb9ba3c33207c7f77b/kahua.host.uno/Assets/images/kai.scale-150.png',
  'httpVersion': 'HTTP/1.1',
  'headers': [{'name': 'Host', 'value': 'devweeklylaunch.kahua.com'},
   {'name': 'User-Agent',
    'value': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:144.0) Gecko/20100101 Firefox/144.0'},
   {'name': 'Accept',
    'value': 'image/avif,image/webp,image/png,image/svg+xml,image/*;q=0.8,*/*;q=0.5'},
   {'name': 'Accept-Language', 'value': 'en-US,en;q=0.5'},
   {'name': 'Accept-Encoding', 'value': 'gzip, deflate, br, zstd'},
   {'name': 'Referer', 'value': 'https://devweeklylaunch.kahua.com/'},
   {'name': 'Connection', 'value': 'keep-alive'},
   {'name': 'Cookie',
    'value': '_ga_LDSR0PG7K1=GS2.1.s1761055881$o7$g0$t1761056361$j60$l0$h0; _ga=GA1.2.1797269329.1748525036; __q_state_GYCtoSR5LQAFT45X=eyJ1dWlkIjoiZDYyMGI2

In [ ]:
import json
from pprint import pprint

# ---- Paste your blob into `data` (as a Python dict). If you have it as a string, do: data = json.loads(the_string) ----
data = {
    "PropertyName": "StartProjectInfo",
    "Id": "0",
    "Menu": "{\"Ref\":null,\"EntityActionListName\":null,\"MenuItems\":[{\"PreActions\":[],\"Action\":{\"Ref\":\"\",\"TargetType\":7,\"TargetResolutionType\":1,\"Target\":\"\",\"Operation\":13,\"Parameters\":[{\"Name\":\"AppName\",\"DefaultValue\":\"\",\"Value\":\"kahua_AEC_ASI\",\"ValueDateTime\":\"0001-01-01T00:00:00\",\"ComparisonType\":0,\"When\":[],\"DataType\":26,\"Path\":\"\"}],\"Confirmation\":null,\"CompleteNotification\":null,\"Scope\":7,\"ScopeTarget\":\"\",\"Events\":[],\"TargetWhere\":[],\"HostScript\":{\"Script\":\"\",\"Parameters\":[],\"MethodRedirects\":[]},\"HostHooks\":[],\"CommandActions\":[],\"IsEmpty\":false,\"IsProtected\":false,\"AppHostType\":\"Default\",\"Name\":\"\",\"ExtensionMode\":0,\"WorkflowDefName\":\"\",\"AutomationId\":\"\"},\"AppName\":\"kahua_AEC_ASI\",\"Enabled\":true,\"EnabledWhen\":[],\"HasPermission\":true,\"Image\":\"\",\"Key\":\"\",\"Label\":\"Architect's Supplemental Instructions\",\"MenuItems\":[],\"Name\":\"App\",\"Opacity\":0.0,\"Parameters\":\"\",\"ParentMenu\":\"kahua_AEC_ASI.AppMainMenu\",\"Permission\":null,\"Ref\":\"\",\"Type\":0,\"Visible\":true,\"VisibleWhen\":[],\"ValidateOnSubmit\":false,\"EntityActionListName\":\"\",\"EntityActionListSource\":\"\",\"FlyoutSettings\":null,\"AutomationId\":\"\"},{\"PreActions\":[],\"Action\":{\"Ref\":\"\",\"TargetType\":7,\"TargetResolutionType\":1,\"Target\":\"\",\"Operation\":13,\"Parameters\":[{\"Name\":\"AppName\",\"DefaultValue\":\"\",\"Value\":\"kahua_AEC_DesignReviewComments\",\"ValueDateTime\":\"0001-01-01T00:00:00\",\"ComparisonType\":0,\"When\":[],\"DataType\":26,\"Path\":\"\"}],\"Confirmation\":null,\"CompleteNotification\":null,\"Scope\":7,\"ScopeTarget\":\"\",\"Events\":[],\"TargetWhere\":[],\"HostScript\":{\"Script\":\"\",\"Parameters\":[],\"MethodRedirects\":[]},\"HostHooks\":[],\"CommandActions\":[],\"IsEmpty\":false,\"IsProtected\":false,\"AppHostType\":\"Default\",\"Name\":\"\",\"ExtensionMode\":0,\"WorkflowDefName\":\"\",\"AutomationId\":\"\"},\"AppName\":\"kahua_AEC_DesignReviewComments\",\"Enabled\":true,\"EnabledWhen\":[],\"HasPermission\":true,\"Image\":\"\",\"Key\":\"\",\"Label\":\"Design Review Comments\",\"MenuItems\":[],\"Name\":\"App\",\"Opacity\":0.0,\"Parameters\":\"\",\"ParentMenu\":\"kahua_AEC_DesignReviewComments.AppMainMenu\",\"Permission\":null,\"Ref\":\"\",\"Type\":0,\"Visible\":true,\"VisibleWhen\":[],\"ValidateOnSubmit\":false,\"EntityActionListName\":\"\",\"EntityActionListSource\":\"\",\"FlyoutSettings\":null,\"AutomationId\":\"\"},{\"PreActions\":[],\"Action\":{\"Ref\":\"\",\"TargetType\":7,\"TargetResolutionType\":1,\"Target\":\"\",\"Operation\":13,\"Parameters\":[{\"Name\":\"AppName\",\"DefaultValue\":\"\",\"Value\":\"kahua_AEC_PunchList_Template\",\"ValueDateTime\":\"0001-01-01T00:00:00\",\"ComparisonType\":0,\"When\":[],\"DataType\":26,\"Path\":\"\"}],\"Confirmation\":null,\"CompleteNotification\":null,\"Scope\":7,\"ScopeTarget\":\"\",\"Events\":[],\"TargetWhere\":[],\"HostScript\":{\"Script\":\"\",\"Parameters\":[],\"MethodRedirects\":[]},\"HostHooks\":[],\"CommandActions\":[],\"IsEmpty\":false,\"IsProtected\":false,\"AppHostType\":\"Default\",\"Name\":\"\",\"ExtensionMode\":0,\"WorkflowDefName\":\"\",\"AutomationId\":\"\"},\"AppName\":\"kahua_AEC_PunchList_Template\",\"Enabled\":true,\"EnabledWhen\":[],\"HasPermission\":true,\"Image\":\"\",\"Key\":\"\",\"Label\":\"kahua_AEC_PunchList_Template\",\"MenuItems\":[],\"Name\":\"App\",\"Opacity\":0.0,\"Parameters\":\"\",\"ParentMenu\":\"kahua_AEC_PunchList_Template.AppMainMenu\",\"Permission\":null,\"Ref\":\"\",\"Type\":0,\"Visible\":true,\"VisibleWhen\":[],\"ValidateOnSubmit\":false,\"EntityActionListName\":\"\",\"EntityActionListSource\":\"\",\"FlyoutSettings\":null,\"AutomationId\":\"\"},{\"PreActions\":[],\"Action\":{\"Ref\":\"\",\"TargetType\":7,\"TargetResolutionType\":1,\"Target\":\"\",\"Operation\":13,\"Parameters\":[{\"Name\":\"AppName\",\"DefaultValue\":\"\",\"Value\":\"kahua_ComplianceTracking\",\"ValueDateTime\":\"0001-01-01T00:00:00\",\"ComparisonType\":0,\"When\":[],\"DataType\":26,\"Path\":\"\"}],\"Confirmation\":null,\"CompleteNotification\":null,\"Scope\":7,\"ScopeTarget\":\"\",\"Events\":[],\"TargetWhere\":[],\"HostScript\":{\"Script\":\"\",\"Parameters\":[],\"MethodRedirects\":[]},\"HostHooks\":[],\"CommandActions\":[],\"IsEmpty\":false,\"IsProtected\":false,\"AppHostType\":\"Default\",\"Name\":\"\",\"ExtensionMode\":0,\"WorkflowDefName\":\"\",\"AutomationId\":\"\"},\"AppName\":\"kahua_ComplianceTracking\",\"Enabled\":true,\"EnabledWhen\":[],\"HasPermission\":true,\"Image\":\"\",\"Key\":\"\",\"Label\":\"Compliance Tracking\",\"MenuItems\":[],\"Name\":\"App\",\"Opacity\":0.0,\"Parameters\":\"\",\"ParentMenu\":\"kahua_ComplianceTracking.AppMainMenu\",\"Permission\":null,\"Ref\":\"\",\"Type\":0,\"Visible\":true,\"VisibleWhen\":[],\"ValidateOnSubmit\":false,\"EntityActionListName\":\"\",\"EntityActionListSource\":\"\",\"FlyoutSettings\":null,\"AutomationId\":\"\"},{\"PreActions\":[],\"Action\":{\"Ref\":\"\",\"TargetType\":7,\"TargetResolutionType\":1,\"Target\":\"\",\"Operation\":13,\"Parameters\":[{\"Name\":\"AppName\",\"DefaultValue\":\"\",\"Value\":\"kahua_DocumentTypes\",\"ValueDateTime\":\"0001-01-01T00:00:00\",\"ComparisonType\":0,\"When\":[],\"DataType\":26,\"Path\":\"\"}],\"Confirmation\":null,\"CompleteNotification\":null,\"Scope\":7,\"ScopeTarget\":\"\",\"Events\":[],\"TargetWhere\":[],\"HostScript\":{\"Script\":\"\",\"Parameters\":[],\"MethodRedirects\":[]},\"HostHooks\":[],\"CommandActions\":[],\"IsEmpty\":false,\"IsProtected\":false,\"AppHostType\":\"Default\",\"Name\":\"\",\"ExtensionMode\":0,\"WorkflowDefName\":\"\",\"AutomationId\":\"\"},\"AppName\":\"kahua_DocumentTypes\",\"Enabled\":true,\"EnabledWhen\":[],\"HasPermission\":true,\"Image\":\"\",\"Key\":\"\",\"Label\":\"Document Types\",\"MenuItems\":[],\"Name\":\"App\",\"Opacity\":0.0,\"Parameters\":\"\",\"ParentMenu\":\"kahua_DocumentTypes.AppMainMenu\",\"Permission\":null,\"Ref\":\"\",\"Type\":0,\"Visible\":true,\"VisibleWhen\":[],\"ValidateOnSubmit\":false,\"EntityActionListName\":\"\",\"EntityActionListSource\":\"\",\"FlyoutSettings\":null,\"AutomationId\":\"\"},{\"PreActions\":[],\"Action\":{\"Ref\":\"\",\"TargetType\":7,\"TargetResolutionType\":1,\"Target\":\"\",\"Operation\":13,\"Parameters\":[{\"Name\":\"AppName\",\"DefaultValue\":\"\",\"Value\":\"kahua_EmployeeProfile\",\"ValueDateTime\":\"0001-01-01T00:00:00\",\"ComparisonType\":0,\"When\":[],\"DataType\":26,\"Path\":\"\"}],\"Confirmation\":null,\"CompleteNotification\":null,\"Scope\":7,\"ScopeTarget\":\"\",\"Events\":[],\"TargetWhere\":[],\"HostScript\":{\"Script\":\"\",\"Parameters\":[],\"MethodRedirects\":[]},\"HostHooks\":[],\"CommandActions\":[],\"IsEmpty\":false,\"IsProtected\":false,\"AppHostType\":\"Default\",\"Name\":\"\",\"ExtensionMode\":0,\"WorkflowDefName\":\"\",\"AutomationId\":\"\"},\"AppName\":\"kahua_EmployeeProfile\",\"Enabled\":true,\"EnabledWhen\":[],\"HasPermission\":true,\"Image\":\"\",\"Key\":\"\",\"Label\":\"Employee Profile\",\"MenuItems\":[],\"Name\":\"App\",\"Opacity\":0.0,\"Parameters\":\"\",\"ParentMenu\":\"kahua_EmployeeProfile.AppMainMenu\",\"Permission\":null,\"Ref\":\"\",\"Type\":0,\"Visible\":true,\"VisibleWhen\":[],\"ValidateOnSubmit\":false,\"EntityActionListName\":\"\",\"EntityActionListSource\":\"\",\"FlyoutSettings\":null,\"AutomationId\":\"\"},{\"PreActions\":[],\"Action\":{\"Ref\":\"\",\"TargetType\":7,\"TargetResolutionType\":1,\"Target\":\"\",\"Operation\":13,\"Parameters\":[{\"Name\":\"AppName\",\"DefaultValue\":\"\",\"Value\":\"kahua_MaterialsCatalog\",\"ValueDateTime\":\"0001-01-01T00:00:00\",\"ComparisonType\":0,\"When\":[],\"DataType\":26,\"Path\":\"\"}],\"Confirmation\":null,\"CompleteNotification\":null,\"Scope\":7,\"ScopeTarget\":\"\",\"Events\":[],\"TargetWhere\":[],\"HostScript\":{\"Script\":\"\",\"Parameters\":[],\"MethodRedirects\":[]},\"HostHooks\":[],\"CommandActions\":[],\"IsEmpty\":false,\"IsProtected\":false,\"AppHostType\":\"Default\",\"Name\":\"\",\"ExtensionMode\":0,\"WorkflowDefName\":\"\",\"AutomationId\":\"\"},\"AppName\":\"kahua_MaterialsCatalog\",\"Enabled\":true,\"EnabledWhen\":[],\"HasPermission\":true,\"Image\":\"\",\"Key\":\"\",\"Label\":\"Materials Catalog\",\"MenuItems\":[],\"Name\":\"App\",\"Opacity\":0.0,\"Parameters\":\"\",\"ParentMenu\":\"kahua_MaterialsCatalog.AppMainMenu\",\"Permission\":null,\"Ref\":\"\",\"Type\":0,\"Visible\":true,\"VisibleWhen\":[],\"ValidateOnSubmit\":false,\"EntityActionListName\":\"\",\"EntityActionListSource\":\"\",\"FlyoutSettings\":null,\"AutomationId\":\"\"},{\"PreActions\":[],\"Action\":{\"Ref\":\"\",\"TargetType\":7,\"TargetResolutionType\":1,\"Target\":\"\",\"Operation\":13,\"Parameters\":[{\"Name\":\"AppName\",\"DefaultValue\":\"\",\"Value\":\"kahua_OmniClassCode\",\"ValueDateTime\":\"0001-01-01T00:00:00\",\"ComparisonType\":0,\"When\":[],\"DataType\":26,\"Path\":\"\"}],\"Confirmation\":null,\"CompleteNotification\":null,\"Scope\":7,\"ScopeTarget\":\"\",\"Events\":[],\"TargetWhere\":[],\"HostScript\":{\"Script\":\"\",\"Parameters\":[],\"MethodRedirects\":[]},\"HostHooks\":[],\"CommandActions\":[],\"IsEmpty\":false,\"IsProtected\":false,\"AppHostType\":\"Default\",\"Name\":\"\",\"ExtensionMode\":0,\"WorkflowDefName\":\"\",\"AutomationId\":\"\"},\"AppName\":\"kahua_OmniClassCode\",\"Enabled\":true,\"EnabledWhen\":[],\"HasPermission\":true,\"Image\":\"\",\"Key\":\"\",\"Label\":\"OmniClass Codes\",\"MenuItems\":[],\"Name\":\"App\",\"Opacity\":0.0,\"Parameters\":\"\",\"ParentMenu\":\"kahua_OmniClassCode.AppMainMenu\",\"Permission\":null,\"Ref\":\"\",\"Type\":0,\"Visible\":true,\"VisibleWhen\":[],\"ValidateOnSubmit\":false,\"EntityActionListName\":\"\",\"EntityActionListSource\":\"\",\"FlyoutSettings\":null,\"AutomationId\":\"\"},{\"PreActions\":[],\"Action\":{\"Ref\":\"\",\"TargetType\":7,\"TargetResolutionType\":1,\"Target\":\"\",\"Operation\":14,\"Parameters\":[{\"Name\":\"AppGroupName\",\"DefaultValue\":\"\",\"Value\":\"kahua_ProjectDirectory.ProjectDirectory\",\"ValueDateTime\":\"0001-01-01T00:00:00\",\"ComparisonType\":0,\"When\":[],\"DataType\":26,\"Path\":\"\"}],\"Confirmation\":null,\"CompleteNotification\":null,\"Scope\":7,\"ScopeTarget\":\"\",\"Events\":[],\"TargetWhere\":[],\"HostScript\":{\"Script\":\"\",\"Parameters\":[],\"MethodRedirects\":[]},\"HostHooks\":[],\"CommandActions\":[],\"IsEmpty\":false,\"IsProtected\":false,\"AppHostType\":\"Default\",\"Name\":\"\",\"ExtensionMode\":0,\"WorkflowDefName\":\"\",\"AutomationId\":\"\"},\"AppName\":\"kahua_ProjectDirectory\",\"Enabled\":true,\"EnabledWhen\":[],\"HasPermission\":true,\"Image\":\"\",\"Key\":\"\",\"Label\":\"Project Directory\",\"MenuItems\":[],\"Name\":\"ProjectDirectory\",\"Opacity\":100.0,\"Parameters\":\"\",\"ParentMenu\":\"kahua_ProjectDirectory.AppMainMenu\",\"Permission\":null,\"Ref\":\"\",\"Type\":3,\"Visible\":true,\"VisibleWhen\":[],\"ValidateOnSubmit\":true,\"EntityActionListName\":\"\",\"EntityActionListSource\":\"\",\"FlyoutSettings\":null,\"AutomationId\":\"\"},{\"PreActions\":[],\"Action\":{\"Ref\":\"\",\"TargetType\":7,\"TargetResolutionType\":1,\"Target\":\"\",\"Operation\":13,\"Parameters\":[{\"Name\":\"AppName\",\"DefaultValue\":\"\",\"Value\":\"kahua_ProjectSharing\",\"ValueDateTime\":\"0001-01-01T00:00:00\",\"ComparisonType\":0,\"When\":[],\"DataType\":26,\"Path\":\"\"}],\"Confirmation\":null,\"CompleteNotification\":null,\"Scope\":7,\"ScopeTarget\":\"\",\"Events\":[],\"TargetWhere\":[],\"HostScript\":{\"Script\":\"\",\"Parameters\":[],\"MethodRedirects\":[]},\"HostHooks\":[],\"CommandActions\":[],\"IsEmpty\":false,\"IsProtected\":false,\"AppHostType\":\"Default\",\"Name\":\"\",\"ExtensionMode\":0,\"WorkflowDefName\":\"\",\"AutomationId\":\"\"},\"AppName\":\"kahua_ProjectSharing\",\"Enabled\":true,\"EnabledWhen\":[],\"HasPermission\":true,\"Image\":\"\",\"Key\":\"\",\"Label\":\"Project Sharing\",\"MenuItems\":[],\"Name\":\"App\",\"Opacity\":0.0,\"Parameters\":\"\",\"ParentMenu\":\"kahua_ProjectSharing.AppMainMenu\",\"Permission\":null,\"Ref\":\"\",\"Type\":0,\"Visible\":true,\"VisibleWhen\":[],\"ValidateOnSubmit\":false,\"EntityActionListName\":\"\",\"EntityActionListSource\":\"\",\"FlyoutSettings\":null,\"AutomationId\":\"\"},{\"PreActions\":[],\"Action\":{\"Ref\":\"\",\"TargetType\":7,\"TargetResolutionType\":1,\"Target\":\"\",\"Operation\":13,\"Parameters\":[{\"Name\":\"AppName\",\"DefaultValue\":\"\",\"Value\":\"kahua_QRCodes\",\"ValueDateTime\":\"0001-01-01T00:00:00\",\"ComparisonType\":0,\"When\":[],\"DataType\":26,\"Path\":\"\"}],\"Confirmation\":null,\"CompleteNotification\":null,\"Scope\":7,\"ScopeTarget\":\"\",\"Events\":[],\"TargetWhere\":[],\"HostScript\":{\"Script\":\"\",\"Parameters\":[],\"MethodRedirects\":[]},\"HostHooks\":[],\"CommandActions\":[],\"IsEmpty\":false,\"IsProtected\":false,\"AppHostType\":\"Default\",\"Name\":\"\",\"ExtensionMode\":0,\"WorkflowDefName\":\"\",\"AutomationId\":\"\"},\"AppName\":\"kahua_QRCodes\",\"Enabled\":true,\"EnabledWhen\":[],\"HasPermission\":true,\"Image\":\"\",\"Key\":\"\",\"Label\":\"kahua_QRCodes\",\"MenuItems\":[],\"Name\":\"App\",\"Opacity\":0.0,\"Parameters\":\"\",\"ParentMenu\":\"kahua_QRCodes.AppMainMenu\",\"Permission\":null,\"Ref\":\"\",\"Type\":0,\"Visible\":true,\"VisibleWhen\":[],\"ValidateOnSubmit\":false,\"EntityActionListName\":\"\",\"EntityActionListSource\":\"\",\"FlyoutSettings\":null,\"AutomationId\":\"\"},{\"PreActions\":[],\"Action\":{\"Ref\":\"\",\"TargetType\":7,\"TargetResolutionType\":1,\"Target\":\"\",\"Operation\":13,\"Parameters\":[{\"Name\":\"AppName\",\"DefaultValue\":\"\",\"Value\":\"kahua_SupplementalCode\",\"ValueDateTime\":\"0001-01-01T00:00:00\",\"ComparisonType\":0,\"When\":[],\"DataType\":26,\"Path\":\"\"}],\"Confirmation\":null,\"CompleteNotification\":null,\"Scope\":7,\"ScopeTarget\":\"\",\"Events\":[],\"TargetWhere\":[],\"HostScript\":{\"Script\":\"\",\"Parameters\":[],\"MethodRedirects\":[]},\"HostHooks\":[],\"CommandActions\":[],\"IsEmpty\":false,\"IsProtected\":false,\"AppHostType\":\"Default\",\"Name\":\"\",\"ExtensionMode\":0,\"WorkflowDefName\":\"\",\"AutomationId\":\"\"},\"AppName\":\"kahua_SupplementalCode\",\"Enabled\":true,\"EnabledWhen\":[],\"HasPermission\":true,\"Image\":\"\",\"Key\":\"\",\"Label\":\"Supplemental Codes\",\"MenuItems\":[],\"Name\":\"App\",\"Opacity\":0.0,\"Parameters\":\"\",\"ParentMenu\":\"kahua_SupplementalCode.AppMainMenu\",\"Permission\":null,\"Ref\":\"\",\"Type\":0,\"Visible\":true,\"VisibleWhen\":[],\"ValidateOnSubmit\":false,\"EntityActionListName\":\"\",\"EntityActionListSource\":\"\",\"FlyoutSettings\":null,\"AutomationId\":\"\"},{\"PreActions\":[],\"Action\":{\"Ref\":\"\",\"TargetType\":7,\"TargetResolutionType\":1,\"Target\":\"\",\"Operation\":13,\"Parameters\":[{\"Name\":\"AppName\",\"DefaultValue\":\"\",\"Value\":\"kahua_WorkPackage\",\"ValueDateTime\":\"0001-01-01T00:00:00\",\"ComparisonType\":0,\"When\":[],\"DataType\":26,\"Path\":\"\"}],\"Confirmation\":null,\"CompleteNotification\":null,\"Scope\":7,\"ScopeTarget\":\"\",\"Events\":[],\"TargetWhere\":[],\"HostScript\":{\"Script\":\"\",\"Parameters\":[],\"MethodRedirects\":[]},\"HostHooks\":[],\"CommandActions\":[],\"IsEmpty\":false,\"IsProtected\":false,\"AppHostType\":\"Default\",\"Name\":\"\",\"ExtensionMode\":0,\"WorkflowDefName\":\"\",\"AutomationId\":\"\"},\"AppName\":\"kahua_WorkPackage\",\"Enabled\":true,\"EnabledWhen\":[],\"HasPermission\":true,\"Image\":\"\",\"Key\":\"\",\"Label\":\"Work Packages\",\"MenuItems\":[],\"Name\":\"App\",\"Opacity\":0.0,\"Parameters\":\"\",\"ParentMenu\":\"kahua_WorkPackage.AppMainMenu\",\"Permission\":null,\"Ref\":\"\",\"Type\":0,\"Visible\":true,\"VisibleWhen\":[],\"ValidateOnSubmit\":false,\"EntityActionListName\":\"\",\"EntityActionListSource\":\"\",\"FlyoutSettings\":null,\"AutomationId\":\"\"}],\"Permission\":null,\"Label\":null,\"Description\":null,\"ParentMenu\":null,\"CompiledScripts\":[],\"IsProtected\":false,\"AppHostType\":null,\"Name\":\"MainMenu\",\"ExtensionMode\":0,\"WorkflowDefName\":null,\"AutomationId\":null}",
    "Info": "{\"canAdd\":true,\"canCopy\":true,\"canCopyOrAddAtOrBelow\":true,\"hasTemplates\":false,\"hideMyProjectsLocationColumn\":false,\"hideMyProjectsStatusColumn\":false,\"id\":0,\"partitionId\":-1,\"name\":\"AWrightCo\",\"number\":\"\",\"path\":\"AWrightCo\",\"isArchived\":false,\"showGettingStartedAction\":false,\"projectDirectoryEnforced\":false,\"childCount\":0,\"isOrganizationProject\":false,\"organizationParentId\":0,\"organizationParentPath\":null}",
    "AppOperationPermissions": "{\"OperationNames\":[\"kahua_License.Grant\",\"kahua_License.Purchase\",\"kahua_License.Revoke\",\"kahua_License.View\",\"kahua_UserManagement.Add\",\"kahua_UserManagement.Edit\",\"kahua_UserManagement.Remove\",\"kahua_UserManagement.Revoke\",\"kahua_UserManagement.View\",\"kahua_UserManagement.Configure\",\"kahua_AEC_PunchList_Libraries.Add\",\"kahua_AEC_PunchList_Libraries.Configure\",\"kahua_AEC_PunchList_Libraries.CreateList\",\"kahua_AEC_PunchList_Libraries.Delete\",\"kahua_AEC_PunchList_Libraries.Edit\",\"kahua_AEC_PunchList_Libraries.Export\",\"kahua_AEC_PunchList_Libraries.Import\",\"kahua_AEC_PunchList_Libraries.OpenApp\",\"kahua_AEC_PunchList_Libraries.Send\",\"kahua_AEC_PunchList_Libraries.Share\",\"kahua_AEC_PunchList_Libraries.View\",\"kahua_AEC_PunchList_MasterLibrary.Add\",\"kahua_AEC_PunchList_MasterLibrary.Configure\",\"kahua_AEC_PunchList_MasterLibrary.CreateList\",\"kahua_AEC_PunchList_MasterLibrary.Delete\",\"kahua_AEC_PunchList_MasterLibrary.Edit\",\"kahua_AEC_PunchList_MasterLibrary.Export\",\"kahua_AEC_PunchList_MasterLibrary.Import\",\"kahua_AEC_PunchList_MasterLibrary.OpenApp\",\"kahua_AEC_PunchList_MasterLibrary.Send\",\"kahua_AEC_PunchList_MasterLibrary.Share\",\"kahua_AEC_PunchList_MasterLibrary.View\",\"kahua_AEC_PunchList_Template.Add\",\"kahua_AEC_PunchList_Template.Configure\",\"kahua_AEC_PunchList_Template.CreateList\",\"kahua_AEC_PunchList_Template.Delete\",\"kahua_AEC_PunchList_Template.Edit\",\"kahua_AEC_PunchList_Template.Export\",\"kahua_AEC_PunchList_Template.Import\",\"kahua_AEC_PunchList_Template.OpenApp\",\"kahua_AEC_PunchList_Template.Send\",\"kahua_AEC_PunchList_Template.Share\",\"kahua_AEC_PunchList_Template.View\",\"kahua_AEC_PunchListSuite.Add\",\"kahua_AEC_PunchListSuite.Configure\",\"kahua_AEC_PunchListSuite.Delete\",\"kahua_AEC_PunchListSuite.Edit\",\"kahua_AEC_PunchListSuite.Export\",\"kahua_AEC_PunchListSuite.Import\",\"kahua_AEC_PunchListSuite.OpenApp\",\"kahua_AEC_PunchListSuite.Send\",\"kahua_AEC_PunchListSuite.View\",\"kahua_CompanyManager.Add\",\"kahua_CompanyManager.Delete\",\"kahua_CompanyManager.Edit\",\"kahua_CompanyManager.CommunityAdd\",\"kahua_CompanyManager.Export\",\"kahua_CompanyManager.Import\",\"kahua_CompanyManager.OpenApp\",\"kahua_CompanyManager.View\",\"kahua_CompanyManager.Configure\",\"kahua_DeletedList.Delete\",\"kahua_DeletedList.Restore\",\"kahua_DeletedList.View\",\"kahua_DomainCompany.Edit\",\"kahua_DomainSettings.Edit\",\"kahua_EmployeeProfile.Add\",\"kahua_EmployeeProfile.Configure\",\"kahua_EmployeeProfile.CreateList\",\"kahua_EmployeeProfile.Delete\",\"kahua_EmployeeProfile.Edit\",\"kahua_EmployeeProfile.Export\",\"kahua_EmployeeProfile.Import\",\"kahua_EmployeeProfile.OpenApp\",\"kahua_EmployeeProfile.Send\",\"kahua_EmployeeProfile.View\",\"kahua_MaterialsCatalog.Add\",\"kahua_MaterialsCatalog.Configure\",\"kahua_MaterialsCatalog.CreateList\",\"kahua_MaterialsCatalog.Delete\",\"kahua_MaterialsCatalog.Edit\",\"kahua_MaterialsCatalog.Export\",\"kahua_MaterialsCatalog.Import\",\"kahua_MaterialsCatalog.OpenApp\",\"kahua_MaterialsCatalog.Send\",\"kahua_MaterialsCatalog.View\",\"kahua_OmniClassCode.Add\",\"kahua_OmniClassCode.Configure\",\"kahua_OmniClassCode.CreateList\",\"kahua_OmniClassCode.CreateReport\",\"kahua_OmniClassCode.Delete\",\"kahua_OmniClassCode.Edit\",\"kahua_OmniClassCode.Export\",\"kahua_OmniClassCode.History\",\"kahua_OmniClassCode.Import\",\"kahua_OmniClassCode.OpenApp\",\"kahua_OmniClassCode.Reports\",\"kahua_OmniClassCode.View\",\"kahua_PeopleManager.Add\",\"kahua_PeopleManager.Delete\",\"kahua_PeopleManager.Edit\",\"kahua_PeopleManager.CommunityAdd\",\"kahua_PeopleManager.Export\",\"kahua_PeopleManager.Import\",\"kahua_PeopleManager.OpenApp\",\"kahua_PeopleManager.View\",\"kahua_PeopleManager.Configure\",\"kahua_ProductServiceClassification.Add\",\"kahua_ProductServiceClassification.Configure\",\"kahua_ProductServiceClassification.CreateList\",\"kahua_ProductServiceClassification.Delete\",\"kahua_ProductServiceClassification.Edit\",\"kahua_ProductServiceClassification.Export\",\"kahua_ProductServiceClassification.Import\",\"kahua_ProductServiceClassification.OpenApp\",\"kahua_ProductServiceClassification.Send\",\"kahua_ProductServiceClassification.Share\",\"kahua_ProductServiceClassification.View\",\"kahua_RecycleBin.Delete\",\"kahua_RecycleBin.Restore\",\"kahua_RecycleBin.View\",\"kahua_SharingInbound.Edit\",\"kahua_SharingInbound.Remove\",\"kahua_SharingInbound.View\",\"kahua_SharingInbound.Configure\",\"kahua_SharingOutbound.Add\",\"kahua_SharingOutbound.Edit\",\"kahua_SharingOutbound.Remove\",\"kahua_SharingOutbound.Revoke\",\"kahua_SharingOutbound.View\",\"kahua_SharingOutbound.Configure\",\"kahua_SystemOfMeasurement.Add\",\"kahua_SystemOfMeasurement.Delete\",\"kahua_SystemOfMeasurement.Edit\",\"kahua_SystemOfMeasurement.Export\",\"kahua_SystemOfMeasurement.Import\",\"kahua_SystemOfMeasurement.OpenApp\",\"kahua_SystemOfMeasurement.View\",\"kahua_SystemOfMeasurement.CreateReport\",\"kahua_SystemOfMeasurement.Reports\",\"kahua_SystemOfMeasurement.Configure\",\"kahua_AEC_ASI.Add\",\"kahua_AEC_ASI.Configure\",\"kahua_AEC_ASI.CreateList\",\"kahua_AEC_ASI.Delete\",\"kahua_AEC_ASI.Edit\",\"kahua_AEC_ASI.Export\",\"kahua_AEC_ASI.Import\",\"kahua_AEC_ASI.OpenApp\",\"kahua_AEC_ASI.Send\",\"kahua_AEC_ASI.Share\",\"kahua_AEC_ASI.View\",\"kahua_AEC_ASI.Copy\",\"kahua_AEC_Communications.Add\",\"kahua_AEC_Communications.Configure\",\"kahua_AEC_Communications.CreateList\",\"kahua_AEC_Communications.Delete\",\"kahua_AEC_Communications.Edit\",\"kahua_AEC_Communications.OpenApp\",\"kahua_AEC_Communications.Send\",\"kahua_AEC_Communications.View\",\"kahua_AEC_Communications.Share\",\"kahua_AEC_Communications.EditViewTemplate\",\"kahua_AEC_Communications.Process\",\"kahua_AEC_CSICode.Add\",\"kahua_AEC_CSICode.Configure\",\"kahua_AEC_CSICode.Delete\",\"kahua_AEC_CSICode.Edit\",\"kahua_AEC_CSICode.Export\",\"kahua_AEC_CSICode.Import\",\"kahua_AEC_CSICode.OpenApp\",\"kahua_AEC_CSICode.View\",\"kahua_AEC_DailyReport.Add\",\"kahua_AEC_DailyReport.Configure\",\"kahua_AEC_DailyReport.CreateList\",\"kahua_AEC_DailyReport.Edit\",\"kahua_AEC_DailyReport.Export\",\"kahua_AEC_DailyReport.Import\",\"kahua_AEC_DailyReport.OpenApp\",\"kahua_AEC_DailyReport.Remove\",\"kahua_AEC_DailyReport.Share\",\"kahua_AEC_DailyReport.View\",\"kahua_AEC_DailyReport.Send\",\"kahua_AEC_DailyReport.Process\",\"kahua_AEC_DailyReport.CreateReport\",\"kahua_AEC_DailyReport.Copy\",\"kahua_AEC_DailyReport.EditViewTemplate\",\"kahua_AEC_DesignReviewComments.Add\",\"kahua_AEC_DesignReviewComments.Configure\",\"kahua_AEC_DesignReviewComments.CreateList\",\"kahua_AEC_DesignReviewComments.Delete\",\"kahua_AEC_DesignReviewComments.Edit\",\"kahua_AEC_DesignReviewComments.Export\",\"kahua_AEC_DesignReviewComments.Import\",\"kahua_AEC_DesignReviewComments.OpenApp\",\"kahua_AEC_DesignReviewComments.Send\",\"kahua_AEC_DesignReviewComments.Share\",\"kahua_AEC_DesignReviewComments.View\",\"kahua_AEC_DesignReviewSets.Add\",\"kahua_AEC_DesignReviewSets.Configure\",\"kahua_AEC_DesignReviewSets.CreateList\",\"kahua_AEC_DesignReviewSets.Delete\",\"kahua_AEC_DesignReviewSets.Edit\",\"kahua_AEC_DesignReviewSets.Export\",\"kahua_AEC_DesignReviewSets.Import\",\"kahua_AEC_DesignReviewSets.OpenApp\",\"kahua_AEC_DesignReviewSets.Send\",\"kahua_AEC_DesignReviewSets.Share\",\"kahua_AEC_DesignReviewSets.View\",\"kahua_AEC_FieldObservation.Add\",\"kahua_AEC_FieldObservation.Configure\",\"kahua_AEC_FieldObservation.CreateList\",\"kahua_AEC_FieldObservation.Edit\",\"kahua_AEC_FieldObservation.Export\",\"kahua_AEC_FieldObservation.Import\",\"kahua_AEC_FieldObservation.Remove\",\"kahua_AEC_FieldObservation.View\",\"kahua_AEC_FieldObservation.Send\",\"kahua_AEC_FieldObservation.CreateReport\",\"kahua_AEC_FieldObservation.Share\",\"kahua_AEC_FieldObservation.OpenApp\",\"kahua_AEC_FieldObservation.Process\",\"kahua_AEC_FieldObservation.EditViewTemplate\",\"kahua_AEC_Invoice.Add\",\"kahua_AEC_Invoice.Configure\",\"kahua_AEC_Invoice.CreateList\",\"kahua_AEC_Invoice.Delete\",\"kahua_AEC_Invoice.Edit\",\"kahua_AEC_Invoice.Export\",\"kahua_AEC_Invoice.Import\",\"kahua_AEC_Invoice.OpenApp\",\"kahua_AEC_Invoice.Send\",\"kahua_AEC_Invoice.Share\",\"kahua_AEC_Invoice.View\",\"kahua_AEC_Invoice.EditViewTemplate\",\"kahua_AEC_PackagedSubmittals.Configure\",\"kahua_AEC_PunchList.Add\",\"kahua_AEC_PunchList.Configure\",\"kahua_AEC_PunchList.CreateList\",\"kahua_AEC_PunchList.Delete\",\"kahua_AEC_PunchList.Edit\",\"kahua_AEC_PunchList.Export\",\"kahua_AEC_PunchList.Import\",\"kahua_AEC_PunchList.OpenApp\",\"kahua_AEC_PunchList.Send\",\"kahua_AEC_PunchList.Share\",\"kahua_AEC_PunchList.View\",\"kahua_AEC_PunchList.Process\",\"kahua_AEC_PunchList.EditViewTemplate\",\"kahua_AEC_PunchList_ProjectLibrary.Add\",\"kahua_AEC_PunchList_ProjectLibrary.Configure\",\"kahua_AEC_PunchList_ProjectLibrary.CreateList\",\"kahua_AEC_PunchList_ProjectLibrary.Delete\",\"kahua_AEC_PunchList_ProjectLibrary.Edit\",\"kahua_AEC_PunchList_ProjectLibrary.Export\",\"kahua_AEC_PunchList_ProjectLibrary.Import\",\"kahua_AEC_PunchList_ProjectLibrary.OpenApp\",\"kahua_AEC_PunchList_ProjectLibrary.Send\",\"kahua_AEC_PunchList_ProjectLibrary.Share\",\"kahua_AEC_PunchList_ProjectLibrary.View\",\"kahua_AEC_Quote.Add\",\"kahua_AEC_Quote.Configure\",\"kahua_AEC_Quote.CreateList\",\"kahua_AEC_Quote.Delete\",\"kahua_AEC_Quote.Edit\",\"kahua_AEC_Quote.Export\",\"kahua_AEC_Quote.Import\",\"kahua_AEC_Quote.OpenApp\",\"kahua_AEC_Quote.Send\",\"kahua_AEC_Quote.Share\",\"kahua_AEC_Quote.View\",\"kahua_AEC_RFI.Add\",\"kahua_AEC_RFI.Configure\",\"kahua_AEC_RFI.CreateList\",\"kahua_AEC_RFI.Edit\",\"kahua_AEC_RFI.Export\",\"kahua_AEC_RFI.Import\",\"kahua_AEC_RFI.OpenApp\",\"kahua_AEC_RFI.Remove\",\"kahua_AEC_RFI.Share\",\"kahua_AEC_RFI.View\",\"kahua_AEC_RFI.Send\",\"kahua_AEC_RFI.Process\",\"kahua_AEC_RFI.EditViewTemplate\",\"kahua_AEC_Shipment.Add\",\"kahua_AEC_Shipment.Configure\",\"kahua_AEC_Shipment.CreateList\",\"kahua_AEC_Shipment.Delete\",\"kahua_AEC_Shipment.Edit\",\"kahua_AEC_Shipment.Export\",\"kahua_AEC_Shipment.Import\",\"kahua_AEC_Shipment.OpenApp\",\"kahua_AEC_Shipment.Send\",\"kahua_AEC_Shipment.Share\",\"kahua_AEC_Shipment.View\",\"kahua_AEC_Sub_RFI.OpenApp\",\"kahua_AEC_Sub_RFI.Add\",\"kahua_AEC_Sub_RFI.Reports\",\"kahua_AEC_Sub_RFI.CreateReport\",\"kahua_AEC_Sub_RFI.Export\",\"kahua_AEC_Sub_RFI.Import\",\"kahua_AEC_Sub_RFI.Share\",\"kahua_AEC_Sub_RFI.Send\",\"kahua_AEC_Sub_RFI.Edit\",\"kahua_AEC_Sub_RFI.View\",\"kahua_AEC_Sub_RFI.Process\",\"kahua_AEC_Sub_RFI.Delete\",\"kahua_AEC_Sub_RFI.History\",\"kahua_AEC_Sub_RFI.Configure\",\"kahua_AEC_Sub_RFI.CreateList\",\"kahua_AEC_Sub_RFI.EditViewTemplate\",\"kahua_AEC_SubGC_RFI.Add\",\"kahua_AEC_SubGC_RFI.Configure\",\"kahua_AEC_SubGC_RFI.CreateList\",\"kahua_AEC_SubGC_RFI.Delete\",\"kahua_AEC_SubGC_RFI.Edit\",\"kahua_AEC_SubGC_RFI.Export\",\"kahua_AEC_SubGC_RFI.Import\",\"kahua_AEC_SubGC_RFI.Process\",\"kahua_AEC_SubGC_RFI.OpenApp\",\"kahua_AEC_SubGC_RFI.Send\",\"kahua_AEC_SubGC_RFI.Share\",\"kahua_AEC_SubGC_RFI.View\",\"kahua_AEC_SubGC_RFI.History\",\"kahua_AEC_SubGC_RFI.CreateReport\",\"kahua_AEC_SubGC_RFI.Reports\",\"kahua_AEC_SubGC_RFI.EditViewTemplate\",\"kahua_AEC_Submittal.Add\",\"kahua_AEC_Submittal.Configure\",\"kahua_AEC_Submittal.CreateList\",\"kahua_AEC_Submittal.Edit\",\"kahua_AEC_Submittal.Export\",\"kahua_AEC_Submittal.Import\",\"kahua_AEC_Submittal.Remove\",\"kahua_AEC_Submittal.Share\",\"kahua_AEC_Submittal.View\",\"kahua_AEC_Submittal.Send\",\"kahua_AEC_Submittal.OpenApp\",\"kahua_AEC_Submittal.Process\",\"kahua_AEC_SubmittalItem.Add\",\"kahua_AEC_SubmittalItem.Configure\",\"kahua_AEC_SubmittalItem.CreateList\",\"kahua_AEC_SubmittalItem.Edit\",\"kahua_AEC_SubmittalItem.Export\",\"kahua_AEC_SubmittalItem.Import\",\"kahua_AEC_SubmittalItem.Remove\",\"kahua_AEC_SubmittalItem.Share\",\"kahua_AEC_SubmittalItem.View\",\"kahua_AEC_SubmittalItem.History\",\"kahua_AEC_SubmittalItem.Send\",\"kahua_AEC_SubmittalItem.OpenApp\",\"kahua_AEC_SubmittalItem.Process\",\"kahua_AEC_SubmittalItem.EditViewTemplate\",\"kahua_AEC_SubmittalPackage.Add\",\"kahua_AEC_SubmittalPackage.Configure\",\"kahua_AEC_SubmittalPackage.CreateList\",\"kahua_AEC_SubmittalPackage.Edit\",\"kahua_AEC_SubmittalPackage.Export\",\"kahua_AEC_SubmittalPackage.Import\",\"kahua_AEC_SubmittalPackage.Remove\",\"kahua_AEC_SubmittalPackage.Share\",\"kahua_AEC_SubmittalPackage.View\",\"kahua_AEC_SubmittalPackage.History\",\"kahua_AEC_SubmittalPackage.Send\",\"kahua_AEC_SubmittalPackage.OpenApp\",\"kahua_AEC_SubmittalPackage.EditViewTemplate\",\"kahua_BudgetAdjustment.Add\",\"kahua_BudgetAdjustment.Configure\",\"kahua_BudgetAdjustment.CreateList\",\"kahua_BudgetAdjustment.Delete\",\"kahua_BudgetAdjustment.Edit\",\"kahua_BudgetAdjustment.Export\",\"kahua_BudgetAdjustment.Import\",\"kahua_BudgetAdjustment.OpenApp\",\"kahua_BudgetAdjustment.Send\",\"kahua_BudgetAdjustment.Share\",\"kahua_BudgetAdjustment.View\",\"kahua_BudgetAdjustment.Process\",\"kahua_BudgetAdjustment.EditViewTemplate\",\"kahua_CalculationTemplate.Configure\",\"kahua_Calendar.Add\",\"kahua_Calendar.Configure\",\"kahua_Calendar.Delete\",\"kahua_Calendar.Edit\",\"kahua_Calendar.OpenApp\",\"kahua_Calendar.Print\",\"kahua_Calendar.Share\",\"kahua_Catalog.Add\",\"kahua_Catalog.Configure\",\"kahua_Catalog.CreateList\",\"kahua_Catalog.Delete\",\"kahua_Catalog.Edit\",\"kahua_Catalog.Export\",\"kahua_Catalog.Import\",\"kahua_Catalog.OpenApp\",\"kahua_Catalog.Send\",\"kahua_Catalog.Share\",\"kahua_Catalog.View\",\"kahua_ClientContract.Add\",\"kahua_ClientContract.Configure\",\"kahua_ClientContract.CreateList\",\"kahua_ClientContract.Delete\",\"kahua_ClientContract.Edit\",\"kahua_ClientContract.Export\",\"kahua_ClientContract.Import\",\"kahua_ClientContract.OpenApp\",\"kahua_ClientContract.Send\",\"kahua_ClientContract.Share\",\"kahua_ClientContract.View\",\"kahua_ClientContract.Process\",\"kahua_ClientContract.EditViewTemplate\",\"kahua_ClientContractChangeOrder.Add\",\"kahua_ClientContractChangeOrder.Configure\",\"kahua_ClientContractChangeOrder.CreateList\",\"kahua_ClientContractChangeOrder.Delete\",\"kahua_ClientContractChangeOrder.Edit\",\"kahua_ClientContractChangeOrder.Export\",\"kahua_ClientContractChangeOrder.Import\",\"kahua_ClientContractChangeOrder.OpenApp\",\"kahua_ClientContractChangeOrder.Send\",\"kahua_ClientContractChangeOrder.Share\",\"kahua_ClientContractChangeOrder.View\",\"kahua_ClientContractChangeOrder.Process\",\"kahua_ClientContractChangeOrder.EditViewTemplate\",\"kahua_ClientContractInvoice.Add\",\"kahua_ClientContractInvoice.Configure\",\"kahua_ClientContractInvoice.CreateList\",\"kahua_ClientContractInvoice.Delete\",\"kahua_ClientContractInvoice.Edit\",\"kahua_ClientContractInvoice.Export\",\"kahua_ClientContractInvoice.Import\",\"kahua_ClientContractInvoice.OpenApp\",\"kahua_ClientContractInvoice.Send\",\"kahua_ClientContractInvoice.Share\",\"kahua_ClientContractInvoice.View\",\"kahua_ComplianceTracking.Add\",\"kahua_ComplianceTracking.Configure\",\"kahua_ComplianceTracking.CreateList\",\"kahua_ComplianceTracking.Delete\",\"kahua_ComplianceTracking.Edit\",\"kahua_ComplianceTracking.Export\",\"kahua_ComplianceTracking.Import\",\"kahua_ComplianceTracking.OpenApp\",\"kahua_ComplianceTracking.Send\",\"kahua_ComplianceTracking.Share\",\"kahua_ComplianceTracking.View\",\"kahua_Configuration.OpenApp\",\"kahua_Configuration.EditApplications\",\"kahua_Configuration.EditAutomation\",\"kahua_Configuration.EditArchiving\",\"kahua_Configuration.EditCurrency\",\"kahua_Configuration.EditApprovals\",\"kahua_Configuration.EditSettings\",\"kahua_Configuration.EditWeather\",\"kahua_Configuration.EditAuthorization\",\"kahua_Configuration.EditInvitationTemplates\",\"kahua_Configuration.EditStampTemplates\",\"kahua_Configuration.EditMarkupColor\",\"kahua_Configuration.EditCalendar\",\"kahua_Configuration.EditCost\",\"kahua_Configuration.AddClients\",\"kahua_Configuration.EditClients\",\"kahua_Configuration.AssignClients\",\"kahua_Configuration.EditDomainInvitation\",\"kahua_Contract.Add\",\"kahua_Contract.Configure\",\"kahua_Contract.CreateList\",\"kahua_Contract.Delete\",\"kahua_Contract.Edit\",\"kahua_Contract.Export\",\"kahua_Contract.Import\",\"kahua_Contract.OpenApp\",\"kahua_Contract.Send\",\"kahua_Contract.Share\",\"kahua_Contract.View\",\"kahua_Contract.Process\",\"kahua_Contract.EditViewTemplate\",\"kahua_ContractChangeOrder.Add\",\"kahua_ContractChangeOrder.Configure\",\"kahua_ContractChangeOrder.CreateList\",\"kahua_ContractChangeOrder.Delete\",\"kahua_ContractChangeOrder.Edit\",\"kahua_ContractChangeOrder.Export\",\"kahua_ContractChangeOrder.Import\",\"kahua_ContractChangeOrder.OpenApp\",\"kahua_ContractChangeOrder.Send\",\"kahua_ContractChangeOrder.Share\",\"kahua_ContractChangeOrder.View\",\"kahua_ContractChangeOrder.Process\",\"kahua_ContractChangeOrder.EditViewTemplate\",\"kahua_ContractChangeRequest.Add\",\"kahua_ContractChangeRequest.Configure\",\"kahua_ContractChangeRequest.CreateList\",\"kahua_ContractChangeRequest.Delete\",\"kahua_ContractChangeRequest.Edit\",\"kahua_ContractChangeRequest.Export\",\"kahua_ContractChangeRequest.Import\",\"kahua_ContractChangeRequest.OpenApp\",\"kahua_ContractChangeRequest.Send\",\"kahua_ContractChangeRequest.Share\",\"kahua_ContractChangeRequest.View\",\"kahua_ContractChangeRequest.Process\",\"kahua_ContractChangeRequest.EditViewTemplate\",\"kahua_ContractInvoice.Add\",\"kahua_ContractInvoice.Configure\",\"kahua_ContractInvoice.CreateList\",\"kahua_ContractInvoice.Delete\",\"kahua_ContractInvoice.Edit\",\"kahua_ContractInvoice.Export\",\"kahua_ContractInvoice.Import\",\"kahua_ContractInvoice.OpenApp\",\"kahua_ContractInvoice.Send\",\"kahua_ContractInvoice.Share\",\"kahua_ContractInvoice.View\",\"kahua_ContractInvoice.EditViewTemplate\",\"kahua_Dashboard.Add\",\"kahua_Dashboard.Configure\",\"kahua_Dashboard.Delete\",\"kahua_Dashboard.Edit\",\"kahua_Dashboard.OpenApp\",\"kahua_Dashboard.Print\",\"kahua_Dashboard.Share\",\"kahua_DocumentTypes.Add\",\"kahua_DocumentTypes.Configure\",\"kahua_DocumentTypes.CreateList\",\"kahua_DocumentTypes.Delete\",\"kahua_DocumentTypes.Edit\",\"kahua_DocumentTypes.Export\",\"kahua_DocumentTypes.Import\",\"kahua_DocumentTypes.OpenApp\",\"kahua_DocumentTypes.Send\",\"kahua_DocumentTypes.Share\",\"kahua_DocumentTypes.View\",\"kahua_FileManager.None\",\"kahua_FileManager.AddFile\",\"kahua_FileManager.AddFolder\",\"kahua_FileManager.Configure\",\"kahua_FileManager.DeleteFile\",\"kahua_FileManager.DeleteFileContributor\",\"kahua_FileManager.DeleteFolder\",\"kahua_FileManager.Download\",\"kahua_FileManager.EditFile\",\"kahua_FileManager.EditFolder\",\"kahua_FileManager.LockUnlock\",\"kahua_FileManager.LockUnlockAll\",\"kahua_FileManager.Markup\",\"kahua_FileManager.OpenApp\",\"kahua_FileManager.SendFile\",\"kahua_FileManager.ViewFile\",\"kahua_FileManager.ViewFolder\",\"kahua_FileManager.Share\",\"kahua_FileManager.CreateReport\",\"kahua_FileManager.SendFileForSecureSignature\",\"kahua_FileManager.SyncFolder\",\"kahua_FileManager.Process\",\"kahua_FileManager.AddDocument\",\"kahua_FileManager.DrawingLogExport\",\"kahua_FileManager.DrawingLogImport\",\"kahua_FileManager.ControlledQAAccept\",\"kahua_FileManager.ControlledQAReject\",\"kahua_FileManager.ControlledRelease\",\"kahua_FileManager.ControlledSupersede\",\"kahua_FileManager.MoveFile\",\"kahua_FundingBudget.Add\",\"kahua_FundingBudget.Configure\",\"kahua_FundingBudget.CreateList\",\"kahua_FundingBudget.Delete\",\"kahua_FundingBudget.Edit\",\"kahua_FundingBudget.Export\",\"kahua_FundingBudget.Import\",\"kahua_FundingBudget.OpenApp\",\"kahua_FundingBudget.Send\",\"kahua_FundingBudget.Share\",\"kahua_FundingBudget.View\",\"kahua_FundingBudget.EditViewTemplate\",\"kahua_FundingChangeRequest.Add\",\"kahua_FundingChangeRequest.Configure\",\"kahua_FundingChangeRequest.CreateList\",\"kahua_FundingChangeRequest.Delete\",\"kahua_FundingChangeRequest.Edit\",\"kahua_FundingChangeRequest.Export\",\"kahua_FundingChangeRequest.Import\",\"kahua_FundingChangeRequest.OpenApp\",\"kahua_FundingChangeRequest.Send\",\"kahua_FundingChangeRequest.Share\",\"kahua_FundingChangeRequest.View\",\"kahua_GroupManagement.Add\",\"kahua_GroupManagement.Edit\",\"kahua_GroupManagement.OpenApp\",\"kahua_GroupManagement.Remove\",\"kahua_GroupManagement.View\",\"kahua_GroupManagement.EditMembers\",\"kahua_Invitation.Add\",\"kahua_Invitation.Configure\",\"kahua_Invitation.Edit\",\"kahua_Invitation.History\",\"kahua_Invitation.OpenApp\",\"kahua_Invitation.Remove\",\"kahua_Invitation.Reports\",\"kahua_Invitation.View\",\"kahua_Issue.Add\",\"kahua_Issue.Configure\",\"kahua_Issue.CreateList\",\"kahua_Issue.Edit\",\"kahua_Issue.Export\",\"kahua_Issue.Import\",\"kahua_Issue.OpenApp\",\"kahua_Issue.Remove\",\"kahua_Issue.Share\",\"kahua_Issue.View\",\"kahua_Issue.Send\",\"kahua_Issue.Process\",\"kahua_Issue.EditViewTemplate\",\"kahua_Location.Add\",\"kahua_Location.Delete\",\"kahua_Location.Edit\",\"kahua_Location.Export\",\"kahua_Location.Import\",\"kahua_Location.OpenApp\",\"kahua_Location.View\",\"kahua_Location.Send\",\"kahua_Location.CreateReport\",\"kahua_Location.Reports\",\"kahua_Location.Configure\",\"kahua_Location.UseDomainLocations\",\"kahua_Location.PromoteToDomain\",\"kahua_MediaManager.Add\",\"kahua_MediaManager.Configure\",\"kahua_MediaManager.Delete\",\"kahua_MediaManager.Download\",\"kahua_MediaManager.Edit\",\"kahua_MediaManager.Markup\",\"kahua_MediaManager.OpenApp\",\"kahua_MediaManager.Send\",\"kahua_MediaManager.Share\",\"kahua_MediaManager.View\",\"kahua_Meeting.Add\",\"kahua_Meeting.Configure\",\"kahua_Meeting.CreateList\",\"kahua_Meeting.Edit\",\"kahua_Meeting.Export\",\"kahua_Meeting.OpenApp\",\"kahua_Meeting.Remove\",\"kahua_Meeting.Share\",\"kahua_Meeting.View\",\"kahua_Meeting.Send\",\"kahua_PlanningAdjustment.Add\",\"kahua_PlanningAdjustment.Configure\",\"kahua_PlanningAdjustment.CreateList\",\"kahua_PlanningAdjustment.Delete\",\"kahua_PlanningAdjustment.Edit\",\"kahua_PlanningAdjustment.Export\",\"kahua_PlanningAdjustment.Import\",\"kahua_PlanningAdjustment.OpenApp\",\"kahua_PlanningAdjustment.Send\",\"kahua_PlanningAdjustment.Share\",\"kahua_PlanningAdjustment.View\",\"kahua_PlanningAdjustment.Process\",\"kahua_PlanningBudget.Add\",\"kahua_PlanningBudget.Configure\",\"kahua_PlanningBudget.CreateList\",\"kahua_PlanningBudget.Delete\",\"kahua_PlanningBudget.Edit\",\"kahua_PlanningBudget.Export\",\"kahua_PlanningBudget.Import\",\"kahua_PlanningBudget.OpenApp\",\"kahua_PlanningBudget.Process\",\"kahua_PlanningBudget.Send\",\"kahua_PlanningBudget.Share\",\"kahua_PlanningBudget.View\",\"kahua_Project.Add\",\"kahua_Project.Configure\",\"kahua_Project.Edit\",\"kahua_Project.OpenApp\",\"kahua_Project.Delete\",\"kahua_Project.View\",\"kahua_Project.Import\",\"kahua_Project.Export\",\"kahua_ProjectDirectory_Companies.Add\",\"kahua_ProjectDirectory_Companies.Configure\",\"kahua_ProjectDirectory_Companies.Edit\",\"kahua_ProjectDirectory_Companies.Remove\",\"kahua_ProjectDirectory_Companies.OpenApp\",\"kahua_ProjectDirectory_Companies.Reports\",\"kahua_ProjectDirectory_Companies.History\",\"kahua_ProjectDirectory_Companies.View\",\"kahua_ProjectDirectory_Contacts.Add\",\"kahua_ProjectDirectory_Contacts.Configure\",\"kahua_ProjectDirectory_Contacts.Edit\",\"kahua_ProjectDirectory_Contacts.Remove\",\"kahua_ProjectDirectory_Contacts.OpenApp\",\"kahua_ProjectDirectory_Contacts.Reports\",\"kahua_ProjectDirectory_Contacts.History\",\"kahua_ProjectDirectory_Contacts.View\",\"kahua_ProjectSharing.Add\",\"kahua_ProjectSharing.Configure\",\"kahua_ProjectSharing.CreateList\",\"kahua_ProjectSharing.Delete\",\"kahua_ProjectSharing.Edit\",\"kahua_ProjectSharing.Export\",\"kahua_ProjectSharing.Import\",\"kahua_ProjectSharing.OpenApp\",\"kahua_ProjectSharing.Send\",\"kahua_ProjectSharing.Share\",\"kahua_ProjectSharing.View\",\"kahua_ProjectTaskManager.OpenApp\",\"kahua_ProjectTaskManager.Edit\",\"kahua_ProjectTaskManager.View\",\"kahua_ProjectTaskManager.Export\",\"kahua_ProjectTaskManager.Process\",\"kahua_ProjectTaskManager.Send\",\"kahua_ProjectTaskManager.Configure\",\"kahua_PurchaseOrder.Add\",\"kahua_PurchaseOrder.Configure\",\"kahua_PurchaseOrder.CreateList\",\"kahua_PurchaseOrder.Delete\",\"kahua_PurchaseOrder.Edit\",\"kahua_PurchaseOrder.Export\",\"kahua_PurchaseOrder.Import\",\"kahua_PurchaseOrder.OpenApp\",\"kahua_PurchaseOrder.Send\",\"kahua_PurchaseOrder.Share\",\"kahua_PurchaseOrder.View\",\"kahua_PurchaseOrder.EditViewTemplate\",\"kahua_QRCodes.Add\",\"kahua_QRCodes.Configure\",\"kahua_QRCodes.CreateList\",\"kahua_QRCodes.Delete\",\"kahua_QRCodes.Edit\",\"kahua_QRCodes.Export\",\"kahua_QRCodes.Import\",\"kahua_QRCodes.OpenApp\",\"kahua_QRCodes.Send\",\"kahua_QRCodes.Share\",\"kahua_QRCodes.View\",\"kahua_QuoteRequest.Add\",\"kahua_QuoteRequest.Configure\",\"kahua_QuoteRequest.CreateList\",\"kahua_QuoteRequest.Delete\",\"kahua_QuoteRequest.Edit\",\"kahua_QuoteRequest.Export\",\"kahua_QuoteRequest.Import\",\"kahua_QuoteRequest.OpenApp\",\"kahua_QuoteRequest.Send\",\"kahua_QuoteRequest.Share\",\"kahua_QuoteRequest.View\",\"kahua_QuoteRequest.EditViewTemplate\",\"kahua_ReportManager.OpenApp\",\"kahua_RiskRegister.Add\",\"kahua_RiskRegister.Configure\",\"kahua_RiskRegister.Remove\",\"kahua_RiskRegister.Edit\",\"kahua_RiskRegister.Export\",\"kahua_RiskRegister.Import\",\"kahua_RiskRegister.OpenApp\",\"kahua_RiskRegister.Send\",\"kahua_RiskRegister.View\",\"kahua_RiskRegister.Share\",\"kahua_RiskRegister.CreateList\",\"kahua_RiskRegister.Process\",\"kahua_SupplementalCode.Add\",\"kahua_SupplementalCode.Configure\",\"kahua_SupplementalCode.Delete\",\"kahua_SupplementalCode.Edit\",\"kahua_SupplementalCode.Export\",\"kahua_SupplementalCode.Import\",\"kahua_SupplementalCode.OpenApp\",\"kahua_SupplementalCode.View\",\"kahua_WorkBreakdown.Add\",\"kahua_WorkBreakdown.Configure\",\"kahua_WorkBreakdown.Delete\",\"kahua_WorkBreakdown.Edit\",\"kahua_WorkBreakdown.Export\",\"kahua_WorkBreakdown.Import\",\"kahua_WorkBreakdown.OpenApp\",\"kahua_WorkBreakdown.View\",\"kahua_WorkBreakdown.Report\",\"kahua_WorkPackage.Add\",\"kahua_WorkPackage.Configure\",\"kahua_WorkPackage.CreateList\",\"kahua_WorkPackage.Delete\",\"kahua_WorkPackage.Edit\",\"kahua_WorkPackage.Export\",\"kahua_WorkPackage.Import\",\"kahua_WorkPackage.OpenApp\",\"kahua_WorkPackage.Send\",\"kahua_WorkPackage.Share\",\"kahua_WorkPackage.View\"],\"GroupNames\":[\"kahua_ApiUser\",\"kahua_PartitionUser\",\"kahua_UserAdministrator\",\"kahua_DomainAdministrator\",\"kahua_DomainUser\"],\"IsDomainAdministrator\":true,\"IsAnalyticsEnabled\":false}",
    "IsOrganizationProject": "False",
    "AccessedFromOrganizationParent": "False",
    "LaunchContainerInfos": {
        "PartitionId": "0",
        "AnalyticsEnabled": "False",
        "_children": [
            {
                "PropertyName": "LaunchContainerInfo",
                "Label": "Cost Management",
                "Name": "CostManagement",
                "TargetInfos": [
                    {
                        "PropertyName": "LaunchTargetInfo",
                        "AppGroup": "kahua_AEC_Funding.Funding",
                        "Name": "kahua_AEC_Funding.fundingLaunch",
                        "AppLabel": "Funding",
                        "AppScope": "Partition",
                        "DisplayMode": "Default",
                        "Icon": "Funding",
                        "IconSvgPathData": "\tM5,6H23V18H5V6M14,9A3,3 0 0,1 17,12A3,3 0 0,1 14,15A3,3 0 0,1 11,12A3,3 0 0,1 14,9M9,8A2,2 0 0,1 7,10V14A2,2 0 0,1 9,16H19A2,2 0 0,1 21,14V10A2,2 0 0,1 19,8H9M1,10H3V20H19V22H1V10Z",
                        "IsOfflineEnabled": "False",
                        "Label": "Funding",
                        "SubApps": "kahua_FundingBudget,kahua_BudgetAdjustment,kahua_ClientContract,kahua_FundingChangeRequest,kahua_ClientContractChangeOrder,kahua_ClientContractInvoice",
                        "SubAppInfos": [
                            {
                                "PropertyName": "LaunchTargetSubAppInfo",
                                "Label": "Budget",
                                "Target": "kahua_FundingBudget"
                            },
                            {
                                "PropertyName": "LaunchTargetSubAppInfo",
                                "Label": "Budget Adjustments",
                                "Target": "kahua_BudgetAdjustment"
                            },
                            {
                                "PropertyName": "LaunchTargetSubAppInfo",
                                "Label": "Contracts",
                                "Target": "kahua_ClientContract"
                            },
                            {
                                "PropertyName": "LaunchTargetSubAppInfo",
                                "Label": "Change Requests",
                                "Target": "kahua_FundingChangeRequest"
                            },
                            {
                                "PropertyName": "LaunchTargetSubAppInfo",
                                "Label": "Change Orders",
                                "Target": "kahua_ClientContractChangeOrder"
                            },
                            {
                                "PropertyName": "LaunchTargetSubAppInfo",
                                "Label": "Pay Apps",
                                "Target": "kahua_ClientContractInvoice"
                            }
                        ]
                    }
                ]
            },
            {
                "PropertyName": "LaunchContainerInfo",
                "Label": "Document Management",
                "Name": "DocumentManagement",
                "TargetInfos": [
                    {
                        "PropertyName": "LaunchTargetInfo",
                        "App": "kahua_RiskRegister",
                        "Name": "kahua_RiskRegister.",
                        "AppLabel": "Risk Register",
                        "AppScope": "Partition",
                        "DisplayMode": "Default",
                        "Icon": "RiskRegister",
                        "IsOfflineEnabled": "False",
                        "Label": "Risk Register"
                    }
                ]
            },
            {
                "PropertyName": "LaunchContainerInfo",
                "Name": "Other",
                "Label": "Other",
                "TargetInfos": [
                    {
                        "PropertyName": "LaunchTargetInfo",
                        "AppGroup": "kahua_ProjectDirectory.ProjectDirectory",
                        "Name": "kahua_ProjectDirectory.projectDirectoryLaunch",
                        "AppLabel": "Project Directory",
                        "AppScope": "Partition",
                        "DisplayMode": "Default",
                        "Icon": "ProjectDirectory",
                        "IconSvgPathData": "\tM15 14C16.33 14 19 14.67 19 16V17H11V16C11 14.67 13.67 14 15 14M15 13C16.11 13 17 12.11 17 11S16.11 9 15 9C13.9 9 13 9.89 13 11C13 12.11 13.9 13 15 13M22 8V18C22 19.11 21.11 20 20 20H4C2.9 20 2 19.11 2 18V6C2 4.89 2.9 4 4 4H10L12 6H20C21.11 6 22 6.9 22 8M20 8H4V18H20V8Z",
                        "IsOfflineEnabled": "False",
                        "Label": "Project Directory",
                        "SubApps": "kahua_ProjectDirectory_Contacts,kahua_ProjectDirectory_Companies",
                        "SubAppInfos": [
                            {
                                "PropertyName": "LaunchTargetSubAppInfo",
                                "Label": "People",
                                "Target": "kahua_ProjectDirectory_Contacts"
                            },
                            {
                                "PropertyName": "LaunchTargetSubAppInfo",
                                "Label": "Companies",
                                "Target": "kahua_ProjectDirectory_Companies"
                            }
                        ]
                    }
                ]
            }
        ]
    }
}

# 1) Decode nested JSON strings -----------------------------------------------
menu = json.loads(data["Menu"]) if isinstance(data.get("Menu"), str) else data.get("Menu", {})
perms = json.loads(data["AppOperationPermissions"]) if isinstance(data.get("AppOperationPermissions"), str) else data.get("AppOperationPermissions", {})

# 2) Top-level apps from Menu.MenuItems ----------------------------------------
menu_apps = []
for item in (menu or {}).get("MenuItems", []):
    # each is a menu "App" entry with AppName + Label
    menu_apps.append({"AppName": item.get("AppName"), "Label": item.get("Label")})

# 3) Launch containers: groups, targets, subapps -------------------------------
containers = data.get("LaunchContainerInfos", {}) or {}
groups = []
for child in containers.get("_children", []):
    label = child.get("Label")
    name  = child.get("Name")
    for tgt in child.get("TargetInfos", []):
        # SubApps is a comma-separated string; SubAppInfos has labels/targets
        subapps = [s.strip() for s in (tgt.get("SubApps") or "").split(",") if s.strip()]
        subapp_infos = [{"Label": si.get("Label"), "Target": si.get("Target")} for si in tgt.get("SubAppInfos", [])]
        groups.append({
            "ContainerLabel": label,
            "ContainerName": name,
            "AppGroup": tgt.get("AppGroup") or tgt.get("App"),
            "AppLabel": tgt.get("AppLabel") or tgt.get("Label"),
            "SubApps": subapps,
            "SubAppInfos": subapp_infos,
        })

# 4) Permissions (flat lists) ---------------------------------------------------
operation_names = (perms or {}).get("OperationNames", [])
group_names = (perms or {}).get("GroupNames", [])

# 5) Unique app names (from Menu + SubApps) ------------------------------------
unique_apps = sorted({*(a["AppName"] for a in menu_apps if a["AppName"]),
                      *([s for g in groups for s in g["SubApps"]])})

print("=== Top-level apps (Menu) ===")
pprint(menu_apps)

print("\n=== Launch Containers / Groups ===")
for g in groups:
    print(f"- {g['ContainerLabel']} ({g['ContainerName']}): AppGroup={g['AppGroup']} Label={g['AppLabel']}")
    if g["SubApps"]:
        print("  SubApps:", ", ".join(g["SubApps"]))
    if g["SubAppInfos"]:
        print("  SubAppInfos:", ", ".join(f"{si['Label']} -> {si['Target']}" for si in g["SubAppInfos"]))

print("\n=== Unique App Names (Menu + SubApps) ===")
for a in unique_apps:
    print(" ", a)

print("\n=== Permission Operation Names (count:", len(operation_names), ") ===")
print(", ".join(operation_names[:20]) + (" ..." if len(operation_names) > 20 else ""))

print("\n=== Permission Groups ===")
print(", ".join(group_names))

=== Top-level apps (Menu) ===
[{'AppName': 'kahua_AEC_ASI', 'Label': "Architect's Supplemental Instructions"},
 {'AppName': 'kahua_AEC_DesignReviewComments',
  'Label': 'Design Review Comments'},
 {'AppName': 'kahua_AEC_PunchList_Template',
  'Label': 'kahua_AEC_PunchList_Template'},
 {'AppName': 'kahua_ComplianceTracking', 'Label': 'Compliance Tracking'},
 {'AppName': 'kahua_DocumentTypes', 'Label': 'Document Types'},
 {'AppName': 'kahua_EmployeeProfile', 'Label': 'Employee Profile'},
 {'AppName': 'kahua_MaterialsCatalog', 'Label': 'Materials Catalog'},
 {'AppName': 'kahua_OmniClassCode', 'Label': 'OmniClass Codes'},
 {'AppName': 'kahua_ProjectDirectory', 'Label': 'Project Directory'},
 {'AppName': 'kahua_ProjectSharing', 'Label': 'Project Sharing'},
 {'AppName': 'kahua_QRCodes', 'Label': 'kahua_QRCodes'},
 {'AppName': 'kahua_SupplementalCode', 'Label': 'Supplemental Codes'},
 {'AppName': 'kahua_WorkPackage', 'Label': 'Work Packages'}]

=== Launch Containers / Groups ===
- Cost Manage

In [7]:
import json

with open('postman2.json', 'r') as f:
    data = json.load(f)

print(json.dumps(data, indent=4))

{
    "data": "AQEAAAABBQEAAAAGAgAAAAdZAAAABAMAAAADAgIHAQQAAAD/BAUAAAADAgID/wQGAAAAAwICA/8EBwAAAAMCAgP/BAgAAAADAgIHAQkAAAD/BAoAAAADAgIHAQsAAAD/BAwAAAADAgID/wQNAAAAAwICBwEOAAAA/wQPAAAAAwICA/8EEAAAAAMCAgcBDgAAAP8EEQAAAAMCAgb/BBIAAAADAgID/wQTAAAAAwICA/8EFAAAAAMCAgP/BBUAAAADAgID/wQWAAAAAwICBwEOAAAA/wQXAAAAAwICAv8EGAAAAAMCAgP/BBkAAAADAgID/wQaAAAAAwICA/8EGwAAAAMCAgEBAP8EHAAAAAMCAgcBDgAAAP8EHQAAAAMCAgEBAP8EHgAAAAMCAgcBHwAAAP8EIAAAAAMCAgP/BCEAAAADAgIHASIAAAD/BCMAAAADAgID/wQkAAAAAwICBwEOAAAA/wQlAAAAAwICBwEOAAAA/wQmAAAAAwICAQEA/wQnAAAAAwICAQEA/wQoAAAAAwICA/8EKQAAAAMCAgL/BCoAAAADAgIC/wQrAAAAAwICAv8ELAAAAAMCAgL/BC0AAAADAgIG/wQuAAAAAwICBwEOAAAA/wQvAAAAAwICAQEA/wQwAAAAAwICAQEA/wQxAAAAAwICBwEOAAAA/wQyAAAAAwICBwEOAAAA/wQzAAAAAwICA/8ENAAAAAMCAgcBDgAAAP8ENQAAAAMCAgP/BDYAAAADAgID/wQ3AAAAAwICA/8EOAAAAAMCAgP/BDkAAAADAgID/wQ6AAAAAwICBwEOAAAA/wQ7AAAAAwICA/8EPAAAAAMCAgcBDgAAAP8EPQAAAAMCAgP/BD4AAAADAgID/wQ/AAAAAwICA/8EQAAAAAMCAgP/BEEAAAADAgID/wRCAAAAAwICA/8EQwAAAAMCAgL/BEQAAAADAgIC/wRFAAAAAwICA/8ERgAAAAMCAgL/B

In [2]:
for key in data['event']:
    print(key)

PropertyName
AppName
PartitionId
DomainPartitionId
ShowTask
ExplicitHubDef
LoadUnreadDiscussionsCount
EventId
Extensions
HubPath
ContextParameters
HostServiceLogs
StepInfo


In [3]:
for key in data['event']['StepInfo']['ViewEvent']:
    print(key)


AppName
DomainPartitionId
PartitionId
LocalContactEntityDataStrings
LocalContactEntityDataPack
DefaultEntityDataStrings
DefaultEntityDataPacks
DomainCode
HubPath
Processors
ViewDef
CostStatusSet
AutoSequenceInfos
PortableViews
Functions
EntityDefMetaData
LookupLists
Send


In [4]:
print(json.dumps(data['event']['StepInfo']['ViewEvent']['EntityDefMetaData'], indent=4))

[
    {
        "PropertyName": "EntityDefMetaData",
        "EntityDef": "kahua_Contract.Contract",
        "IsCostItem": "True",
        "IsDomainPartition": "False",
        "IsDynamicEntityDefinition": "False",
        "EntityDefHierarchy": "a2FodWFfQ29yZS5rYWh1YV9CYXNl,a2FodWFfQ29yZS5rYWh1YV9DaGlsZEJhc2U=,a2FodWFfQ29yZS5rYWh1YV9DdXJyZW5jeVJhdGVFbnRpdHlEZWY=,a2FodWFfQ29zdC5Db3N0SXRlbUJhc2U=,a2FodWFfQ29zdC5Db3N0SXRlbQ==,a2FodWFfQ29zdC5Db3N0SXRlbURvY3VtZW50,a2FodWFfQ29udHJhY3QuQ29udHJhY3Q=",
        "IsMultiProject": "False",
        "CurrencyAttributes": "RnVuZGluZ0l0ZW1zSXRlbXNUb3RhbFZhbHVl,RXhwZW5zZUl0ZW1zSXRlbXNUb3RhbFZhbHVl,RnVuZGluZ0l0ZW1zTWFya3Vwc1RvdGFsVmFsdWU=,RXhwZW5zZUl0ZW1zTWFya3Vwc1RvdGFsVmFsdWU=,QnVkZ2V0Q3VycmVudFRvdGFsVmFsdWU=,QnVkZ2V0Q3VycmVudFVuaXRWYWx1ZQ==,QnVkZ2V0Q3VycmVudFRheEFtb3VudA==,QnVkZ2V0SXRlbXNUb3RhbFRvdGFsVmFsdWU=,Q29zdEN1cnJlbnRUb3RhbFZhbHVl,Q29zdEN1cnJlbnRVbml0VmFsdWU=,Q29zdEN1cnJlbnRUYXhBbW91bnQ=,Q29zdEl0ZW1zVG90YWxUb3RhbFZhbHVl,Q29zdEVzdGltYXRlZFRvdGF

In [ ]:
kahua_types = [
    'kahua_Address.Address',
    'kahua_AEC_Communications.CommunicationsConversation',
    'kahua_AEC_Communications.CommunicationsFax',
    'kahua_AEC_Communications.CommunicationsLetter',
    'kahua_AEC_Communications.CommunicationsMemo',
    'kahua_AEC_Communications.CommunicationsMessage',
    'kahua_AEC_Communications.CommunicationsTransmittal',
    'kahua_AEC_Communications.kahua_Communications',
    'kahua_AEC_Communications.TransmittalItem',
    'kahua_AEC_CSICode.CSICode',
    'kahua_AEC_DailyReport.DailyReport',
    'kahua_AEC_DailyReport.DailyReportCompany',
    'kahua_AEC_DailyReport.DailyReportCompanyLaborType',
    'kahua_AEC_DailyReport.DailyReportEquipment',
    'kahua_AEC_DailyReport.DailyReportFieldObservations',
    'kahua_AEC_DailyReport.DailyReportIncidents',
    'kahua_AEC_DailyReport.DailyReportInternalLaborTracking',
    'kahua_AEC_DailyReport.DailyReportMaterialsReceived',
    'kahua_AEC_DailyReport.DailyReportNotesAndWorkCompleted',
    'kahua_AEC_DailyReport.DailyReportSignature',
    'kahua_AEC_DailyReport.DailyReportVisitors',
    'kahua_AEC_DesignReviewComments.DesignReviewComment',
    'kahua_AEC_DesignReviewComments.ResponseHistoryItem',
    'kahua_AEC_DesignReviewSets.DesignReviewFile',
    'kahua_AEC_DesignReviewSets.DesignReviewFileResponse',
    'kahua_AEC_DesignReviewSets.DesignReviewSet',
    'kahua_AEC_DesignReviewSets.ReviewStatusItem',
    'kahua_AEC_FieldObservation.FieldObservationItem',
    'kahua_AEC_Invoice.Invoice',
    'kahua_AEC_Invoice.InvoiceItem',
    'kahua_AEC_PackagedSubmittals.SubmittalReviewer',
    'kahua_AEC_PunchList.PunchListItem',
    'kahua_AEC_PunchList_Libraries.Defect',
    'kahua_AEC_PunchList_Libraries.Library',
    'kahua_AEC_PunchList_MasterLibrary.Defect',
    'kahua_AEC_PunchList_MasterLibrary.Library',
    'kahua_AEC_PunchList_ProjectLibrary.Defect',
    'kahua_AEC_PunchList_ProjectLibrary.Library',
    'kahua_AEC_PunchList_Template.Template',
    'kahua_AEC_RFI.RFI',
    'kahua_AEC_Sub_RFI.SubRFI',
    'kahua_AEC_SubGC_RFI.SubGCRFI',
    'kahua_AEC_Submittal.SubmittalItem',
    'kahua_AEC_SubmittalItem.SubmittalItem',
    'kahua_AEC_SubmittalItem.SubmittalItemRevision',
    'kahua_AEC_SubmittalPackage.SubmittalPackage',
    'kahua_AEC_SubmittalPackage.SubmittalPackageRevision',
    'kahua_BudgetAdjustment.BudgetAdjustment',
    'kahua_BudgetAdjustment.BudgetAdjustmentItem',
    'kahua_ClientContract.ClientContract',
    'kahua_ClientContract.ClientContractItem',
    'kahua_ClientContractChangeOrder.ClientContractChangeOrder',
    'kahua_ClientContractChangeOrder.ClientContractChangeOrderItem',
    'kahua_ClientContractInvoice.ClientContractInvoice',
    'kahua_ClientContractInvoice.ClientContractInvoiceItem',
    'kahua_CompanyManager.kahua_Certification',
    'kahua_CompanyManager.kahua_Company',
    'kahua_CompanyManager.kahua_Office',
    'kahua_ComplianceTracking.ComplianceTracking',
    'kahua_ComplianceTracking.ComplianceTrackingNotification',
    'kahua_Contract.Contract',
    'kahua_Contract.ContractItem',
    'kahua_ContractChangeOrder.ContractChangeOrder',
    'kahua_ContractChangeOrder.ContractChangeOrderItem',
    'kahua_ContractChangeRequest.ContractChangeRequest',
    'kahua_ContractChangeRequest.ContractChangeRequestItem',
    'kahua_ContractInvoice.ContractInvoice',
    'kahua_ContractInvoice.ContractInvoiceItem',
    'kahua_Core.kahua_AppList',
    'kahua_Core.kahua_ChronologyEntry',
    'kahua_Core.kahua_Comment',
    'kahua_Core.kahua_ConnectTargetBase',
    'kahua_Core.kahua_ExternalReference',
    'kahua_Core.kahua_File',
    'kahua_Core.kahua_FileData',
    'kahua_Core.kahua_Media',
    'kahua_Core.kahua_Pin',
    'kahua_Core.kahua_SelectableItem',
    'kahua_Core.kahua_TagAnchor',
    'kahua_Cost.CostItemStatus',
    'kahua_Cost.CostUnitSetting',
    'kahua_CostItemIndex.CostItemIndex',
    'kahua_CostItemIndex.CostItemIndexConfiguration',
    'kahua_DocumentTypes.DocumentType',
    'kahua_EmployeeProfile.EmployeeProfile',
    'kahua_FileManager.DrawingLog',
    'kahua_FileManager.DrawingLogRevision',
    'kahua_FileManager.File',
    'kahua_FileManager.Folder',
    'kahua_FileManager.Media',
    'kahua_FinancialPeriod.FinancialPeriod',
    'kahua_FinancialPeriod.FinancialPeriodConfiguration',
    'kahua_FinancialPeriod.SettingsConfiguration',
    'kahua_FundingBudget.FundingBudget',
    'kahua_FundingBudget.FundingBudgetItem',
    'kahua_FundingChangeRequest.FundingChangeRequest',
    'kahua_FundingChangeRequest.FundingChangeRequestItem',
    'kahua_Issue.Issue',
    'kahua_Issue.IssueExpenseItem',
    'kahua_Issue.IssueItem',
    'kahua_Location.GeoLocation',
    'kahua_Location.Location',
    'kahua_MaterialsCatalog.MaterialsCatalog',
    'kahua_Meeting.ActionItem',
    'kahua_Meeting.Attendee',
    'kahua_Meeting.Meeting',
    'kahua_Meeting.MeetingItem',
    'kahua_MessageManager.kahua_MessageParticipant',
    'kahua_OmniClassCode.OmniClassCode',
    'kahua_PeopleManager.kahua_Contact',
    'kahua_PlanningAdjustment.PlanningAdjustment',
    'kahua_PlanningAdjustment.PlanningAdjustmentItem',
    'kahua_PlanningBudget.PlanningBudget',
    'kahua_PlanningBudget.PlanningBudgetItem',
    'kahua_Project.Project',
    'kahua_ProjectDirectory_Companies.ProjectDirectoryCompany',
    'kahua_ProjectDirectory_Contacts.ProjectDirectoryContact',
    'kahua_ProjectSharing.ProjectShare',
    'kahua_PurchaseOrder.PurchaseOrder',
    'kahua_PurchaseOrder.PurchaseOrderItem',
    'kahua_QRCodes.QR_Code',
    'kahua_QuoteRequest.QuoteRequest',
    'kahua_QuoteRequest.QuoteRequestItem',
    'kahua_Reference.kahua_CompositeItem',
    'kahua_Reference.kahua_ReferenceBase',
    'kahua_RiskRegister.ActionItem',
    'kahua_RiskRegister.RiskRegister',
    'kahua_RiskRegister.RiskRegisterConfigurationRiskCategory',
    'kahua_SecureSignature.EnvelopeStatus',
    'kahua_SecureSignature.RecipientRoute',
    'kahua_SecureSignature.RecipientStatus',
    'kahua_SecureSignature.SignatureRequest',
    'kahua_Stamp.kahua_BaseStamp',
    'kahua_Stamp.kahua_SignatureStamp',
    'kahua_SupplementalCode.SupplementalCode',
    'kahua_SystemOfMeasurement.UnitType',
    'kahua_SystemOfMeasurement.UnitValue',
    'kahua_Weather.kahua_WeatherConditions',
    'kahua_WorkBreakdown.Item',
    'kahua_WorkBreakdown.WorkBreakdownBudgetItem',
    'kahua_Workflow.kahua_ApprovalResult',
    'kahua_WorkPackage.WorkPackage',
]